*Note: Updated scoring functions have not yet been run for Session 1 surveys*

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive
from openpyxl import load_workbook
import os
!pip install timezonefinder pytz
from timezonefinder import TimezoneFinder
import pytz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.6 MB/s eta 0:00:00


# Set up Environment and Prep Survey Data

In [ ]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)


# Define path to survey data
# Session 1
#s1_data_path = '/content/drive/My Drive/battery_survey_scoring/data/survey/Burnout Battery - Prolific (April 2024)_August 1, 2024_08.30.csv'
s1_data_path = '/content/drive/My Drive/battery_survey_scoring/data/survey/Burnout Battery - Prolific - Session 1 (April 2024)_October 8, 2025_14.15.csv' # new path with new participants 10-8-25
s1_prolific_data_paths = [
    '/content/drive/My Drive/battery_survey_scoring/data/worker_info/prolific_export_s1_b1.csv',
    '/content/drive/My Drive/battery_survey_scoring/data/worker_info/prolific_export_s1_b2.csv',
    '/content/drive/My Drive/battery_survey_scoring/data/worker_info/prolific_export_s1_b3.csv',
    '/content/drive/My Drive/battery_survey_scoring/data/worker_info/prolific_export_s1_b4.csv',
    '/content/drive/My Drive/battery_survey_scoring/data/worker_info/prolific_export_s1_b5.csv',
]

# Session 2
s2_data_path = '/content/drive/My Drive/battery_survey_scoring/data/survey/Burnout Battery - Prolific - Session 2 (2024)_October 8, 2025_14.16.csv' #updated for readi pt 2
s2_prolific_data_paths = [
    '/content/drive/My Drive/battery_survey_scoring/data/worker_info/prolific_export_s2.csv',
    '/content/drive/My Drive/battery_survey_scoring/data/worker_info/prolific_export_s2_b2.csv',
]

# Session 3
s3_data_path = '/content/drive/My Drive/battery_survey_scoring/data/survey/Burnout Battery - Session 3 Pilot (Spring 2025)_October 8, 2025_14.16.csv'
s3_prolific_data_paths = [
    '/content/drive/My Drive/battery_survey_scoring/data/worker_info/prolific_export_s3.csv',
    '/content/drive/My Drive/battery_survey_scoring/data/worker_info/prolific_export_s3_b2.csv',
    ]


# For seeing if they have Ts and Ls data:
# worker_data_path = '/content/drive/My Drive/battery_survey_scoring/data/worker_info/worker_conditions_S2.csv'

# Load survey data
s1_data = pd.read_csv(s1_data_path)
s1_prolific_dfs = [pd.read_csv(path) for path in s1_prolific_data_paths]
s1_prolific_combined_df = pd.concat(s1_prolific_dfs, ignore_index=True)

s2_data = pd.read_csv(s2_data_path)
s2_prolific_dfs = [pd.read_csv(path) for path in s2_prolific_data_paths]
s2_prolific_data = pd.concat(s2_prolific_dfs, ignore_index=True)

s3_data = pd.read_csv(s3_data_path)
s3_prolific_dfs = [pd.read_csv(path) for path in s3_prolific_data_paths]
s3_prolific_data = pd.concat(s3_prolific_dfs, ignore_index=True)

# To load in question text
s1_metadata_row = s1_data.iloc[0]
s2_metadata_row = s2_data.iloc[0]
s3_metadata_row = s3_data.iloc[0]


# Load worker data (Ts and Ls IDs)
#worker_data = pd.read_csv(worker_data_path)
#prolific_ids = set(worker_data['workerId'].tolist())  # Use a set for faster lookup

# Regular expression to filter valid Prolific IDs (assumes Prolific IDs are 24 alphanumeric characters)
valid_id_regex = r'^[a-zA-Z0-9]{24}$'

# IDs to exclude (experimenters' IDs)
exceptions = ['659efaac1d035e0153317cbc']

#**TO DO: filter out participants who do not have prolific data**

# Specify which column to use for prolific_id
# S1
s1_prolific_id = 'Q5'
# S2
s2_prolific_id = 'Q1'
# S3
s3_prolific_id = 'Q5'


# Filter data to only include valid Prolific IDs and exclude specific exceptions
s1_data = s1_data[s1_data[s1_prolific_id].str.match(valid_id_regex, na=False) & ~s1_data[s1_prolific_id].isin(exceptions)]
s2_data = s2_data[s2_data[s2_prolific_id].str.match(valid_id_regex, na=False) & ~s2_data[s2_prolific_id].isin(exceptions)]
s3_data = s3_data[s3_data[s3_prolific_id].str.match(valid_id_regex, na=False) & ~s3_data[s3_prolific_id].isin(exceptions)]


# Add a column to mark if the respondent is in the worker data
#data['has_Ts_and_Ls'] = data[prolific_id].apply(lambda x: 'Yes' if x in prolific_ids else 'No')

# Remove duplicates based on PROLIFIC_PID
s1_filtered_data = s1_data.drop_duplicates(subset=[s1_prolific_id], keep='first')
s2_filtered_data = s2_data.drop_duplicates(subset=[s2_prolific_id], keep='first')
s3_filtered_data = s3_data.drop_duplicates(subset=[s3_prolific_id], keep='first')


#===========================================
# merge prolific data with qualtrics data to keep relevant fields from qualtrics
#===========================================
s1_prolific_combined_df = s1_prolific_combined_df.merge(
    s1_filtered_data[['Q5', 'LocationLatitude', 'LocationLongitude']],
    how='left',
    left_on='Participant id',
    right_on='Q5'
).drop(columns=['Q5'])  # drop Q5 after merge


s2_prolific_data = s2_prolific_data.merge(
    s2_filtered_data[['Q1', 'LocationLatitude', 'LocationLongitude']],
    how='left',
    left_on='Participant id',
    right_on='Q1'
).drop(columns=['Q1'])  # drop Q5 after merge


s3_prolific_data = s3_prolific_data.merge(
    s3_filtered_data[['Q5', 'LocationLatitude', 'LocationLongitude']],
    how='left',
    left_on='Participant id',
    right_on='Q5'
).drop(columns=['Q5'])
#===========================================

# Output the number of unique PROLIFIC_PID in the survey and those matching with worker data
print(f"Total Prolific IDs in Session 1: {s1_data[s1_prolific_id].nunique()}")
print(f"Total unique IDs in S1 filtered data: {s1_filtered_data[s1_prolific_id].nunique()}")

print(f"Total Prolific IDs in survey: {s2_data[s2_prolific_id].nunique()}")
print(f"Total unique IDs in filtered data: {s2_filtered_data[s2_prolific_id].nunique()}")
#print(f"IDs matched with Ts and Ls data: {filtered_data['has_Ts_and_Ls'].value_counts().get('Yes', 0)}")  # Safe access

#Session 3
print(f"Total Prolific IDs in Session 3: {s3_data[s3_prolific_id].nunique()}")
print(f"Total unique IDs in S3 filtered data: {s3_filtered_data[s3_prolific_id].nunique()}")

Mounted at /content/drive
Total Prolific IDs in Session 1: 1701
Total unique IDs in S1 filtered data: 1701
Total Prolific IDs in survey: 1218
Total unique IDs in filtered data: 1218
Total Prolific IDs in Session 3: 779
Total unique IDs in S3 filtered data: 779


In [ ]:
s1_prolific_combined_df['Participant id'].head()

,Participant id
0,65985fb8ca6a586ffeb25e6c
1,659c055cbfece76fed7304eb
2,5c78b032419dcc0014119082
3,5d901b10d7a21a001d635d12
4,65959564477c6a68a50176e4


In [ ]:
pd.set_option('display.max_columns', None)

s1_filtered_data.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q_BallotBoxStuffing,Q5,Time Prolific ID_First Click,Time Prolific ID_Last Click,Time Prolific ID_Page Submit,Time Prolific ID_Click Count,Time - Inf Consent_First Click,Time - Inf Consent_Last Click,Time - Inf Consent_Page Submit,Time - Inf Consent_Click Count,Time - Intro/MturkID_First Click,Time - Intro/MturkID_Last Click,Time - Intro/MturkID_Page Submit,Time - Intro/MturkID_Click Count,Demos_1_1,Demos_1_2,Demos_2,Demos_3,Demos_4,Demos_5,Demos_6_1,Time - Demo1_First Click,Time - Demo1_Last Click,Time - Demo1_Page Submit,Time - Demo1_Click Count,Demos_7,Demos_8,Time - Demo2_First Click,Time - Demo2_Last Click,Time - Demo2_Page Submit,Time - Demo2_Click Count,Demos_9,Demos_10,Demos_11,Time - Demo3_First Click,Time - Demo3_Last Click,Time - Demo3_Page Submit,Time - Demo3_Click Count,Demos_12,Demos_13,Time - Demo4_First Click,Time - Demo4_Last Click,Time - Demo4_Page Submit,Time - Demo4_Click Count,Demos_14_1,Demos_15_1,Demos_16,Demos_17,Demos_18,Time - Demo5_First Click,Time - Demo5_Last Click,Time - Demo5_Page Submit,Time - Demo5_Click Count,Readiness_1,Readiness_2_1,Readiness_3,Readiness_4_1,Readiness_5,Readiness_6,Readiness_7,Readiness_8,Readiness_9,Readiness_10_1,Readiness_11_1,Readiness_12,Readiness_13_1,Readiness_14_1,Readiness_15,Readiness_16,Time - Readiness_First Click,Time - Readiness_Last Click,Time - Readiness_Page Submit,Time - Readiness_Click Count,Burnout_1,Burnout_2,Burnout_3,Burnout_4,Burnout_5,Burnout_6,Burnout_7,Burnout_8,Burnout_9,Burnout_10,Burnout_11,Burnout_12,Burnout_13,Burnout_14,Burnout_15,Burnout_16,Time - Oldenburg_First Click,Time - Oldenburg_Last Click,Time - Oldenburg_Page Submit,Time - Oldenburg_Click Count,BFI_1,BFI_2,BFI_3,BFI_4,BFI_5,BFI_6,BFI_7,BFI_8,BFI_9,BFI_10,BFI_11,BFI_12,BFI_13,BFI_14,BFI_15,BFI_16,BFI_17,BFI_18,BFI_19,BFI_20,BFI_21,BFI_22,BFI_23,BFI_24,BFI_25,BFI_26,BFI_27,BFI_28,BFI_29,BFI_30,BFI_31,BFI_32,BFI_33,BFI_34,BFI_35,BFI_36,BFI_37,BFI_38,BFI_39,BFI_40,BFI_41,BFI_42,BFI_43,BFI_44,Time - Big5_First Click,Time - Big5_Last Click,Time - Big5_Page Submit,Time - Big5_Click Count,Time - AttnCheck 1.1_First Click,Time - AttnCheck 1.1_Last Click,Time - AttnCheck 1.1_Page Submit,Time - AttnCheck 1.1_Click Count,Attn_Check_1_1,Time - AttnCheck 1.2_First Click,Time - AttnCheck 1.2_Last Click,Time - AttnCheck 1.2_Page Submit,Time - AttnCheck 1.2_Click Count,Attn_Check_1_2,Time - AttnCheck 1.3_First Click,Time - AttnCheck 1.3_Last Click,Time - AttnCheck 1.3_Page Submit,Time - AttnCheck 1.3_Click Count,Attn_Check_1_3,Time - AttnCheck 1.4_First Click,Time - AttnCheck 1.4_Last Click,Time - AttnCheck 1.4_Page Submit,Time - AttnCheck 1.4_Click Count,Attn_Check_1_4,ADHD_1,ADHD_2,ADHD_3,ADHD_4,ADHD_5,ADHD_6,ADHD_7,ADHD_8,ADHD_9,ADHD_10,ADHD_11,ADHD_12,ADHD_13,ADHD_14,ADHD_15,ADHD_16,ADHD_17,ADHD_18,ADHD_19,ADHD_20,ADHD_21,ADHD_22,ADHD_23,ADHD_24,Time - ADHD_First Click,Time - ADHD_Last Click,Time - ADHD_Page Submit,Time - ADHD_Click Count,STAI_1_1,STAI_1_2,STAI_1_3,STAI_1_4,STAI_1_5,STAI_1_6,STAI_1_7,STAI_1_8,STAI_1_9,STAI_1_10,STAI_1_11,STAI_1_12,STAI_1_13,STAI_1_14,STAI_1_15,STAI_1_16,STAI_1_17,STAI_1_18,STAI_1_19,STAI_1_20,STAI_2_1,STAI_2_2,STAI_2_3,STAI_2_4,STAI_2_5,STAI_2_6,STAI_2_7,STAI_2_8,STAI_2_9,STAI_2_10,STAI_2_11,STAI_2_12,STAI_2_13,STAI_2_14,STAI_2_15,STAI_2_16,STAI_2_17,STAI_2_18,STAI_2_19,STAI_2_20,Time - STAI_First Click,Time - STAI_Last Click,Time - STAI_Page Submit,Time - STAI_Click Count,Maslach_1,Maslach_2,Maslach_3,Maslach_4,Maslach_5,Maslach_6,Maslach_7,Maslach_8,Maslach_9,Maslach_10,Maslach_11,Maslach_12,Maslach_13,Maslach_14,Maslach_15,Maslach_16,Timing - Maslach _First Click,Timing - Maslach _Last Click,Timing - Maslach _Page Submit,Timing - Maslach _Click Count,Time - AttnCheck 2.1_First Click,Time - AttnCheck 2.1_Last Click,Time - AttnC

In [ ]:
# df_to_check = filtered_data.copy()

# # Define MBI subscale items
# exhaustion_items = [f'Maslach_{i}' for i in [1, 2, 3, 4, 6]]  # Exhaustion (5 items)
# cynicism_items = [f'Maslach_{i}' for i in [8, 9, 13, 14, 15]]  # Cynicism (5 items)
# professional_efficacy_items = [f'Maslach_{i}' for i in [5, 7, 10, 11, 12, 16]]  # Professional Efficacy (6 items)

# # Combine all required MBI columns
# maslach_columns = exhaustion_items + cynicism_items + professional_efficacy_items

# # Calculate the number of missing fields for each person
# df_to_check['Missing_Fields_Count'] = df_to_check[maslach_columns].isna().sum(axis=1)

# # Count the total number of people with any missing fields
# total_people_with_missing_fields = (df_to_check['Missing_Fields_Count'] > 0).sum()

# # Count the total number of people missing all columns
# total_people_missing_all_fields = (df_to_check['Missing_Fields_Count'] == len(maslach_columns)).sum()

# # Get a summary of missing fields for each person
# missing_summary = df_to_check[df_to_check['Missing_Fields_Count'] > 0][['PROLIFIC_PID', 'Missing_Fields_Count']]

# # Print results
# print(f"Total number of people with missing MBI fields: {total_people_with_missing_fields}")
# print(f"Total number of people missing all MBI fields: {total_people_missing_all_fields}")
# print("Details of missing fields for each person:")
# print(missing_summary)


# Define Survey Scoring Functions
Add your scoring code as a function here

## General helper functions

In [ ]:
# Hashing Ids
def hash_id(participant_id):
    """Hashes the participant ID using SHA-256."""
    hash_object = hashlib.sha256()
    hash_object.update(participant_id.encode())
    return hash_object.hexdigest()

## add in some code to generate to column list with prolific ID and hashed ID
s1_hash_match = pd.DataFrame(s1_prolific_combined_df['Participant id'])
s1_hash_match['hashed id'] = s1_prolific_combined_df['Participant id'].apply(hash_id)

s1_hash_match.head()
hashmatchpath = '/content/drive/My Drive/battery_survey_scoring/analyses/J_hashmatch.csv'
s1_hash_match.to_csv(hashmatchpath, index=False)



In [ ]:
def add_metadata_as_multiindex(data, metadata_row):
    """
    Add a metadata row as a secondary header (multi-index for columns).

    Args:
        data (pd.DataFrame): The DataFrame containing the data.
        metadata_row (list or pd.Series): The metadata row to use as a secondary header.

    Returns:
        pd.DataFrame: DataFrame with a multi-index for columns.
    """

    # Ensure metadata_row aligns with data columns
    metadata_df = pd.DataFrame([metadata_row], columns=data.columns)

    # Combine the original column headers and metadata
    multi_index = pd.MultiIndex.from_tuples(
        zip(data.columns, metadata_df.iloc[0]),
        names=["Original Header", "Metadata"]
    )

    # Assign the MultiIndex to the columns
    data.columns = multi_index

    return data


## Session 1 overview tab

In [ ]:

# Compute local start time and end time
# Initialize timezone finder
tf = TimezoneFinder()

# Conversion function
def convert_times_with_timezone(start_utc, end_utc, lat, lon):
    try:
        # Find the timezone name
        tz_name = tf.timezone_at(lat=lat, lng=lon)
        if tz_name is None:
            return pd.NaT, pd.NaT, None

        # Define timezone
        local_tz = pytz.timezone(tz_name)

        # Convert UTC to localized time
        start_local = pd.to_datetime(start_utc, utc=True).astimezone(local_tz)
        end_local = pd.to_datetime(end_utc, utc=True).astimezone(local_tz)

        return start_local, end_local, tz_name
    except:
        return pd.NaT, pd.NaT, None


In [ ]:
# Prolific S1 Overview
def score_overview(df, path):
    column_mapping = {
        'Participant id': 'Subject_ID',
        'Started at': 'StartTime',
        'Time taken': 'Duration',
    }

    df.rename(columns=column_mapping, inplace=True)
    df['Subject_ID'] = df['Subject_ID'].apply(hash_id)

    # Convert seconds to timedelta (NaNs will become NaT)
    df['Duration_HM'] = pd.to_timedelta(df['Duration'], unit='s')

    # Format as H:MM, safely skipping NaTs
    df['Duration_HM'] = df['Duration_HM'].apply(
        lambda x: f"{int(x.total_seconds() // 3600)}:{int((x.total_seconds() % 3600) // 60):02d}"
        if pd.notnull(x) else None
    )

    #add separate start/end date and time
    # Clean column names first
    df.columns = df.columns.str.strip()
    # Convert to datetime only once
    df['StartTime'] = pd.to_datetime(df['StartTime'], utc=True, errors='coerce')
    df['Completed at'] = pd.to_datetime(df['Completed at'], utc=True, errors='coerce')
    # Split into date and time
    df['UTC_start_date'] = df['StartTime'].dt.date
    df['UTC_start_time'] = df['StartTime'].dt.strftime('%H:%M:%S')  # avoids microseconds
    df['UTC_completed_date'] = df['Completed at'].dt.date
    df['UTC_completed_time'] = df['Completed at'].dt.strftime('%H:%M:%S')

    #compute local start and end time
    # Apply conversion
    df[['LocalStartTimestamp', 'LocalEndTimestamp','TimeZone']] = df.apply(
        lambda row: pd.Series(convert_times_with_timezone(
            pd.to_datetime(row["StartTime"]),
            pd.to_datetime(row["Completed at"]),
            pd.to_numeric(row["LocationLatitude"]),
            pd.to_numeric(row["LocationLongitude"])
        )),
        axis=1
    )

    # Split into date and time
    df['local_start_date'] = df['LocalStartTimestamp'].apply(lambda x: x.date() if pd.notna(x) else None)
    df['local_start_time'] = df['LocalStartTimestamp'].apply(lambda x: x.strftime('%H:%M:%S') if pd.notna(x) else None)

    df['local_end_date'] = df['LocalEndTimestamp'].apply(lambda x: x.date() if pd.notna(x) else None)
    df['local_end_time'] = df['LocalEndTimestamp'].apply(lambda x: x.strftime('%H:%M:%S') if pd.notna(x) else None)

    # Define column order: your key columns first, then the rest
    preferred_order = ['Subject_ID', 'UTC_start_time','TimeZone','local_start_time','Duration', 'Duration_HM','local_end_time']
    other_columns = [col for col in df.columns if col not in preferred_order]
    all_columns = preferred_order + other_columns

    # Reorder columns and save
    scored_data = df[all_columns].copy()
    scored_data.to_csv(path, index=False)

    return scored_data


In [ ]:
overview_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_scored_Overview.csv'
scored_overview_data = score_overview(s1_prolific_combined_df, overview_path)
scored_overview_data.head()

,Subject_ID,UTC_start_time,TimeZone,local_start_time,Duration,Duration_HM,local_end_time,Submission id,Status,Custom study tncs accepted at,StartTime,Completed at,Reviewed at,Archived at,Completion code,Total approvals,Age,Sex,Ethnicity simplified,Country of birth,Country of residence,Nationality,Language,Student status,Employment status,LocationLatitude,LocationLongitude,UTC_start_date,UTC_completed_date,UTC_completed_time,LocalStartTimestamp,LocalEndTimestamp,local_start_date,local_end_date
0,be53040ff6aad99b863cf7f3f8e50d29f416b4606a4536...,16:00:38,America/Chicago,11:00:38,4165.0,1:09,12:10:02,6622951a094ac4f560513eef,APPROVED,Not Applicable,2024-04-19 16:00:38.804000+00:00,2024-04-19 17:10:02.980000+00:00,2024-04-20T19:48:07.203000Z,2024-04-19T17:10:03.887204Z,CPWVVLXI,305,24,Female,Asian,United States,United States,United States,English,No,Part-Time,37.6257,-97.3142,2024-04-19,2024-04-19,17:10:02,2024-04-19 11:00:38.804000-05:00,2024-04-19 12:10:02.980000-05:00,2024-04-19,2024-04-19
1,d4d9418fb0eec5ce257fe190e797a78dc5d034baded6a7...,16:01:08,America/New_York,12:01:08,3717.0,1:01,13:03:04,6622953cc6838027d30c66a2,APPROVED,Not Applicable,2024-04-19 16:01:08.048000+00:00,2024-04-19 17:03:04.462000+00:00,2024-04-20T19:48:50.229000Z,2024-04-19T17:03:45.965762Z,CPWVVLXI,297,63,Male,Black,United States,United States,United States,English,Yes,Full-Time,39.7954,-86.2658,2024-04-19,2024-04-19,17:03:04,2024-04-19 12:01:08.048000-04:00,2024-04-19 13:03:04.462000-04:00,2024-04-19,2024-04-19
2,84eaf750fa9c82ffc49185677a168b1ade690ee1946d40...,16:02:47,America/New_York,12:02:47,5042.0,1:24,13:26:48,662295961c2b637cfaffd40e,APPROVED,Not Applicable,2024-04-19 16:02:47.071000+00:00,2024-04-19 17:26:48.547000+00:00,2024-04-20T19:48:51.402000Z,2024-04-19T17:28:10.459274Z,CPWVVLXI,4079,38,Male,White,United States,United States,United States,English,DATA_EXPIRED,DATA_EXPIRED,40.7575,-74.9967,2024-04-19,2024-04-19,17:26:48,2024-04-19 12:02:47.071000-04:00,2024-04-19 13:26:48.547000-04:00,2024-04-19,2024-04-19
3,d5bbb37b5a8d78d00749e85f10df5feb5769db24342eb2...,16:07:55,America/Chicago,11:07:55,2423.0,0:40,11:48:18,66229610df4c42742ec29049,APPROVED,Not Applicable,2024-04-19 16:07:55.521000+00:00,2024-04-19 16:48:18.136000+00:00,2024-04-20T19:48:53.344000Z,2024-04-19T16:48:19.530952Z,CPWVVLXI,1693,33,Male,White,United States,United States,United States,English,DATA_EXPIRED,DATA_EXPIRED,41.5883,-87.4593,2024-04-19,2024-04-19,16:48:18,2024-04-19 11:07:55.521000-05:00,2024-04-19 11:48:18.136000-05:00,2024-04-19,2024-04-19
4,b4ed0383c0d4839b014cdf9ddf0f68197919709e5b6805...,16:05:36,America/Chicago,11:05:36,2637.0,0:43,11:49:32,662296495b6d8c6ae1391297,APPROVED,Not Applicable,2024-04-19 16:05:36.187000+00:00,2024-04-19 16:49:32.216000+00:00,2024-04-20T19:48:52.336000Z,2024-04-19T16:49:41.079800Z,CPWVVLXI,116,25,Female,Black,United States,United States,United States,English,Yes,Full-Time,41.7124,-87.7478,2024-04-19,2024-04-19,16:49:32,2024-04-19 11:05:36.187000-05:00,2024-04-19 11:49:32.216000-05:00,2024-04-19,2024-04-19


## Session 2 Overview

In [ ]:
# Prolific S2 Overview
def score_overview_s2(df, path):
    column_mapping = {
        'Participant id': 'Subject_ID',
        'Started at': 'StartTime',
        'Time taken': 'Duration'
    }

    df.rename(columns=column_mapping, inplace=True)
    df['Subject_ID'] = df['Subject_ID'].apply(hash_id)

    # Convert seconds to timedelta (NaNs will become NaT)
    df['Duration_HM'] = pd.to_timedelta(df['Duration'], unit='s')

    # Format as H:MM, safely skipping NaTs
    df['Duration_HM'] = df['Duration_HM'].apply(
        lambda x: f"{int(x.total_seconds() // 3600)}:{int((x.total_seconds() % 3600) // 60):02d}"
        if pd.notnull(x) else None
    )
    #add separate start/end date and time
    # Clean column names first
    df.columns = df.columns.str.strip()

    # Convert to datetime only once
    df['StartTime'] = pd.to_datetime(df['StartTime'], utc=True, errors='coerce')
    df['Completed at'] = pd.to_datetime(df['Completed at'], utc=True, errors='coerce')
    # Split into date and time
    df['UTC_start_date'] = df['StartTime'].dt.date
    df['UTC_start_time'] = df['StartTime'].dt.strftime('%H:%M:%S')  # avoids microseconds
    df['UTC_completed_date'] = df['Completed at'].dt.date
    df['UTC_completed_time'] = df['Completed at'].dt.strftime('%H:%M:%S')

    #compute local start and end time
    df[['LocalStartTimestamp', 'LocalEndTimestamp','TimeZone']] = df.apply(
        lambda row: pd.Series(convert_times_with_timezone(
            pd.to_datetime(row["StartTime"]),
            pd.to_datetime(row["Completed at"]),
            pd.to_numeric(row["LocationLatitude"]),
            pd.to_numeric(row["LocationLongitude"])
        )),
        axis=1
    )

    # Split into date and time
    df['local_start_date'] = df['LocalStartTimestamp'].apply(lambda x: x.date() if pd.notna(x) else None)
    df['local_start_time'] = df['LocalStartTimestamp'].apply(lambda x: x.strftime('%H:%M:%S') if pd.notna(x) else None)

    df['local_end_date'] = df['LocalEndTimestamp'].apply(lambda x: x.date() if pd.notna(x) else None)
    df['local_end_time'] = df['LocalEndTimestamp'].apply(lambda x: x.strftime('%H:%M:%S') if pd.notna(x) else None)

    # Define column order: your key columns first, then the rest
    preferred_order = ['Subject_ID', 'UTC_start_time','TimeZone','local_start_time','Duration', 'Duration_HM','local_end_time']
    other_columns = [col for col in df.columns if col not in preferred_order]
    all_columns = preferred_order + other_columns

    # Reorder columns and save
    scored_data = df[all_columns].copy()
    scored_data.to_csv(path, index=False)

    return scored_data


In [ ]:
overview_s2_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_s2_scored_Overview.csv'
scored_s2_overview_data = score_overview_s2(s2_prolific_data, overview_s2_path)
scored_s2_overview_data.head()

,Subject_ID,UTC_start_time,TimeZone,local_start_time,Duration,Duration_HM,local_end_time,Submission id,Status,Custom study tncs accepted at,StartTime,Completed at,Reviewed at,Archived at,Completion code,Total approvals,Age,Sex,Ethnicity simplified,Country of birth,Country of residence,Nationality,Language,Student status,Employment status,LocationLatitude,LocationLongitude,UTC_start_date,UTC_completed_date,UTC_completed_time,LocalStartTimestamp,LocalEndTimestamp,local_start_date,local_end_date
0,5491be349322a63b7c89117da4ee0faf02286c5fd842f5...,19:29:03,America/New_York,15:29:03,2137.0,0:35,16:04:39,66b127ee588fe37bc50c59ac,APPROVED,Not Applicable,2024-08-05 19:29:03.395000+00:00,2024-08-05 20:04:39.618000+00:00,2024-08-13T19:02:18.622000Z,2024-08-05T20:04:41.477747Z,C155IWZT,619,41,Female,White,United Kingdom,United States,United States,English,No,Part-Time,27.2636,-82.5171,2024-08-05,2024-08-05,20:04:39,2024-08-05 15:29:03.395000-04:00,2024-08-05 16:04:39.618000-04:00,2024-08-05,2024-08-05
1,e669610ef97f0ca9ba5d806965f18112ffd0414d305067...,20:08:18,America/New_York,16:08:18,1974.0,0:32,16:41:12,66b130eb36d7af3e0a84884b,APPROVED,Not Applicable,2024-08-05 20:08:18.933000+00:00,2024-08-05 20:41:12.748000+00:00,2024-08-13T19:02:19.284000Z,2024-08-05T20:41:13.369588Z,C155IWZT,1554,23,Female,Mixed,United States,United States,United States,English,No,Full-Time,42.1043,-71.6549,2024-08-05,2024-08-05,20:41:12,2024-08-05 16:08:18.933000-04:00,2024-08-05 16:41:12.748000-04:00,2024-08-05,2024-08-05
2,16d2a19f9de26f3a1edb80ad8955cdee1e2e3064de06d8...,20:24:28,America/Los_Angeles,13:24:28,2211.0,0:36,14:01:19,66b134ea5d20af4031b61fc9,APPROVED,Not Applicable,2024-08-05 20:24:28.165000+00:00,2024-08-05 21:01:19.045000+00:00,2024-08-13T19:02:19.703000Z,2024-08-05T21:01:19.583453Z,C155IWZT,2468,32,Male,White,United States,United States,United States,English,DATA_EXPIRED,DATA_EXPIRED,45.5375,-122.5989,2024-08-05,2024-08-05,21:01:19,2024-08-05 13:24:28.165000-07:00,2024-08-05 14:01:19.045000-07:00,2024-08-05,2024-08-05
3,f1243d85b295b6a732788259818b41a7fb85648984e109...,23:55:25,America/New_York,19:55:25,1481.0,0:24,20:20:05,66b1665ecb0528d8b87923aa,APPROVED,Not Applicable,2024-08-05 23:55:25.303000+00:00,2024-08-06 00:20:05.489000+00:00,2024-08-13T19:02:19.996000Z,2024-08-06T00:21:01.175010Z,C155IWZT,1769,32,Male,White,United States,United States,United States,English,DATA_EXPIRED,DATA_EXPIRED,40.782,-73.9953,2024-08-05,2024-08-06,00:20:05,2024-08-05 19:55:25.303000-04:00,2024-08-05 20:20:05.489000-04:00,2024-08-05,2024-08-05
4,cfa469757fbf745cab8d15526dae40b11ab7c74ea96f3d...,19:03:46,America/New_York,15:03:46,2427.0,0:40,15:44:12,66b27392575c0244aa82461d,APPROVED,Not Applicable,2024-08-06 19:03:46.102000+00:00,2024-08-06 19:44:12.268000+00:00,2024-08-13T19:02:20.323000Z,2024-08-06T19:44:12.888845Z,C155IWZT,3831,64,Female,White,United States,United States,United States,English,No,Unemployed (and job seeking),28.6529,-81.2106,2024-08-06,2024-08-06,19:44:12,2024-08-06 15:03:46.102000-04:00,2024-08-06 15:44:12.268000-04:00,2024-08-06,2024-08-06


## Session 3 Overview

In [ ]:
# Prolific S3 Overview
def score_overview_s3(df, path):
    column_mapping = {
        'Participant id': 'Subject_ID',
        'Started at': 'StartTime',
        'Time taken': 'Duration'
    }

    df.rename(columns=column_mapping, inplace=True)
    df['Subject_ID'] = df['Subject_ID'].apply(hash_id)

    # Convert seconds to timedelta (NaNs will become NaT)
    df['Duration_HM'] = pd.to_timedelta(df['Duration'], unit='s')

    # Format as H:MM, safely skipping NaTs
    df['Duration_HM'] = df['Duration_HM'].apply(
        lambda x: f"{int(x.total_seconds() // 3600)}:{int((x.total_seconds() % 3600) // 60):02d}"
        if pd.notnull(x) else None
    )
    #add separate start/end date and time
    # Clean column names first
    df.columns = df.columns.str.strip()

    # Convert to datetime only once
    df['StartTime'] = pd.to_datetime(df['StartTime'], utc=True, errors='coerce')
    df['Completed at'] = pd.to_datetime(df['Completed at'], utc=True, errors='coerce')
    # Split into date and time
    df['UTC_start_date'] = df['StartTime'].dt.date
    df['UTC_start_time'] = df['StartTime'].dt.strftime('%H:%M:%S')  # avoids microseconds
    df['UTC_completed_date'] = df['Completed at'].dt.date
    df['UTC_completed_time'] = df['Completed at'].dt.strftime('%H:%M:%S')

    #compute local start and end time
    df[['LocalStartTimestamp', 'LocalEndTimestamp','TimeZone']] = df.apply(
        lambda row: pd.Series(convert_times_with_timezone(
            pd.to_datetime(row["StartTime"]),
            pd.to_datetime(row["Completed at"]),
            pd.to_numeric(row["LocationLatitude"]),
            pd.to_numeric(row["LocationLongitude"])
        )),
        axis=1
    )

    # Split into date and time
    df['local_start_date'] = df['LocalStartTimestamp'].apply(lambda x: x.date() if pd.notna(x) else None)
    df['local_start_time'] = df['LocalStartTimestamp'].apply(lambda x: x.strftime('%H:%M:%S') if pd.notna(x) else None)

    df['local_end_date'] = df['LocalEndTimestamp'].apply(lambda x: x.date() if pd.notna(x) else None)
    df['local_end_time'] = df['LocalEndTimestamp'].apply(lambda x: x.strftime('%H:%M:%S') if pd.notna(x) else None)

    # Define column order: your key columns first, then the rest
    preferred_order = ['Subject_ID', 'UTC_start_time','TimeZone','local_start_time','Duration', 'Duration_HM','local_end_time']
    other_columns = [col for col in df.columns if col not in preferred_order]
    all_columns = preferred_order + other_columns

    # Reorder columns and save
    scored_data = df[all_columns].copy()
    scored_data.to_csv(path, index=False)

    return scored_data


In [ ]:
overview_s3_path = '/content/drive/My Drive/battery_survey_scoring/analyses/s3_scored_Overview.csv'
scored_s3_overview_data = score_overview_s3(s3_prolific_data, overview_s3_path)
scored_s3_overview_data.head()

,Subject_ID,UTC_start_time,TimeZone,local_start_time,Duration,Duration_HM,local_end_time,Submission id,Status,Custom study tncs accepted at,StartTime,Completed at,Reviewed at,Archived at,Completion code,Total approvals,Age,Sex,Ethnicity simplified,Country of birth,Country of residence,Nationality,Language,Student status,Employment status,LocationLatitude,LocationLongitude,UTC_start_date,UTC_completed_date,UTC_completed_time,LocalStartTimestamp,LocalEndTimestamp,local_start_date,local_end_date
0,86fc1d8847013641c93cd55a837fa60d53bbc47cf9dd9e...,17:58:07,America/Chicago,12:58:07,4172.0,1:09,14:07:38,67ec292f9c94fc1b07c69e89,APPROVED,Not Applicable,2025-04-01 17:58:07.012000+00:00,2025-04-01 19:07:38.128000+00:00,2025-04-07T21:06:26.181000Z,2025-04-01T19:07:39.520466Z,C155IWZT,2250,45,Male,White,United States,United States,United States,English,No,Unemployed (and job seeking),32.9074,-97.4257,2025-04-01,2025-04-01,19:07:38,2025-04-01 12:58:07.012000-05:00,2025-04-01 14:07:38.128000-05:00,2025-04-01,2025-04-01
1,5491be349322a63b7c89117da4ee0faf02286c5fd842f5...,18:14:39,America/New_York,14:14:39,2028.0,0:33,14:48:27,67ec2d013a4e15c62893b5b4,APPROVED,Not Applicable,2025-04-01 18:14:39.923000+00:00,2025-04-01 18:48:27.751000+00:00,2025-04-07T21:06:26.701000Z,2025-04-01T18:48:28.419875Z,C155IWZT,1467,41,Female,White,United Kingdom,United States,United States,English,DATA_EXPIRED,DATA_EXPIRED,27.2636,-82.5171,2025-04-01,2025-04-01,18:48:27,2025-04-01 14:14:39.923000-04:00,2025-04-01 14:48:27.751000-04:00,2025-04-01,2025-04-01
2,4cfed6385f23dfffce02e960dd35c1d841f3c8575c49f7...,18:17:20,America/Los_Angeles,11:17:20,1534.0,0:25,11:42:53,67ec2d93a314c03478d66738,APPROVED,Not Applicable,2025-04-01 18:17:20.203000+00:00,2025-04-01 18:42:53.218000+00:00,2025-04-07T21:06:27.053000Z,2025-04-01T18:42:54.682159Z,C155IWZT,2543,35,Female,Mixed,Saint Helena,United States,United States,English,No,"Not in paid work (e.g. homemaker', 'retired or...",35.4145,-119.0403,2025-04-01,2025-04-01,18:42:53,2025-04-01 11:17:20.203000-07:00,2025-04-01 11:42:53.218000-07:00,2025-04-01,2025-04-01
3,d34667864fcf10241fdc239a8ff71c13679fdff80ad5ec...,23:18:52,America/New_York,19:18:52,1495.0,0:24,19:43:46,67ec7458081c3f989526eb05,APPROVED,Not Applicable,2025-04-01 23:18:52.485000+00:00,2025-04-01 23:43:46.859000+00:00,2025-04-07T21:06:27.385000Z,2025-04-01T23:43:47.661857Z,C155IWZT,8271,38,Male,White,United States,United States,United States,English,No,Part-Time,26.1188,-81.5215,2025-04-01,2025-04-01,23:43:46,2025-04-01 19:18:52.485000-04:00,2025-04-01 19:43:46.859000-04:00,2025-04-01,2025-04-01
4,16d2a19f9de26f3a1edb80ad8955cdee1e2e3064de06d8...,01:27:04,America/Los_Angeles,18:27:04,5774.0,1:36,20:03:17,67ec925d77dea5e0d024dd8d,APPROVED,Not Applicable,2025-04-02 01:27:04.895000+00:00,2025-04-02 03:03:17.963000+00:00,2025-04-07T21:06:27.708000Z,2025-04-02T03:03:18.725073Z,C155IWZT,2992,32,Male,White,United States,United States,United States,English,DATA_EXPIRED,DATA_EXPIRED,45.5136,-122.5946,2025-04-02,2025-04-02,03:03:17,2025-04-01 18:27:04.895000-07:00,2025-04-01 20:03:17.963000-07:00,2025-04-01,2025-04-01


## AQ


In [ ]:
import pandas as pd
import numpy as np

def score_aq(df, path):
    # helper to build item names
    def asq_column(i):
        return f'ASQ_{i}'

    # 5 overall subscores
    default_subscales = {
        'AQ_Social_Skill':        [1,11,13,15,22,36,44,45,47,48],
        'AQ_Attention_Switching': [2, 4,10,16,25,32,34,37,43,46],
        'AQ_Attention_To_Detail': [5, 6, 9,12,19,23,28,29,30,49],
        'AQ_Communication':       [7,17,18,26,27,31,33,35,38,39],
        'AQ_Imagination':         [3, 8,14,20,21,24,40,41,42,50]
    }

    # AQ-Short subscores
    asq_short_subscales = {
        'AQ_Short_Social_Skill':        [1,15,36,45,50],
        'AQ_Short_Routine':             [2,25,34,46],
        'AQ_Short_Switching':           [4,10,32,37],
        'AQ_Short_Imagination':         [3, 8,14,20,36,42,45,50],
        'AQ_Short_Numbers_and_Patterns':[6,9,19,23,41]
    }

    # three-factor subscores
    three_factor_subscales = {
        'AQ_Three_Factor_Sociability':        [7,11,17,28,31,42,50],
        'AQ_Three_Factor_Mentalizing':        [2,15,23,29,30,32],
        'AQ_Three_Factor_Detail_Orientation': [3,5,12,25,26,33,38]
    }

    # macro definitions
    macro_1_questions = [1,2,4,5,6,7,9,12,13,16,18,19,20,21,22,23,26,33,35,39,41,42,43,45,46]
    macro_2_questions = [3,8,10,11,14,15,17,24,25,27,28,29,30,31,32,34,36,37,38,40,44,47,48,49,50]

    # score each ASQ_
    def apply_asq_macros(row):
        out = {}
        for i in macro_1_questions:
            out[asq_column(i)] = int(row[asq_column(i)] in ["Definitely Agree","Slightly Agree"])
        for i in macro_2_questions:
            out[asq_column(i)] = int(row[asq_column(i)] in ["Definitely Disagree","Slightly Disagree"])
        return pd.Series(out)

    # subscale‐sum helper (min_count=1 → all‐NA = NaN)
    def calculate_subscale_scores(df_, subs):
        for name, qs in subs.items():
            cols = [asq_column(i) for i in qs]
            df_[name] = df_[cols].sum(axis=1, min_count=1)

    # flags
    def flag_sparse_data(df_, qcols, min_ans=25):
        return df_[qcols].notna().sum(axis=1) < min_ans
    def flag_zero_std_data(df_, qcols):
        return df_[qcols].std(axis=1, skipna=True) == 0

    # ---- begin scoring ----

    # score the 50 items
    scored_data = df.apply(apply_asq_macros, axis=1)

    # define the 50 columns
    question_columns = [asq_column(i) for i in range(1,51)]

    # blank counts on *original* 50
    scored_data['Blank_Count']      = df[question_columns].isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / len(question_columns)

    # flags
    scored_data['AQ_exclude_sparse']   = flag_sparse_data(scored_data, question_columns)
    scored_data['AQ_exclude_zero_std'] = flag_zero_std_data(scored_data, question_columns)

    # blank out all 50 if excluded
    mask = scored_data['AQ_exclude_sparse'] | scored_data['AQ_exclude_zero_std']
    scored_data.loc[mask, question_columns] = np.nan

    # compute all subscores
    calculate_subscale_scores(scored_data, default_subscales)
    calculate_subscale_scores(scored_data, asq_short_subscales)
    calculate_subscale_scores(scored_data, three_factor_subscales)

    # classic total
    scored_data['AQ_Total_Score'] = scored_data[question_columns].sum(axis=1, min_count=1)

    # SD of the 50 scored items
    scored_data['AQ_Item_SD'] = scored_data[question_columns].std(axis=1, skipna=True)

    # short‐form total (sum of first four AQ-Short subs)
    short_parts = [
        'AQ_Short_Social_Skill',
        'AQ_Short_Routine',
        'AQ_Short_Switching',
        'AQ_Short_Imagination'
    ]
    scored_data['AQ_Short_Total_Score'] = scored_data[short_parts].sum(axis=1, min_count=1)

    # bring in hashed ID
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # reorder exactly as requested
    classic_order = [
        'AQ_Imagination',
        'AQ_Attention_Switching',
        'AQ_Social_Skill',
        'AQ_Communication',
        'AQ_Attention_To_Detail',
        'AQ_Total_Score'
    ]
    three_order = list(three_factor_subscales.keys())
    short_order = [
        'AQ_Short_Social_Skill',
        'AQ_Short_Routine',
        'AQ_Short_Switching',
        'AQ_Short_Imagination',
        'AQ_Short_Total_Score',
        'AQ_Short_Numbers_and_Patterns'
    ]

    final_cols = (
        ['Subject_ID']
      + question_columns
      + classic_order
      + three_order
      + short_order
      + ['AQ_Item_SD','Blank_Count','Blank_Percentage',
         'AQ_exclude_sparse','AQ_exclude_zero_std']
    )
    scored_data = scored_data[final_cols]

    # build the 3-level header so you can see which are reversed
    metadata_texts = s1_metadata_row

    level0 = scored_data.columns
    level1 = [metadata_texts[c] if c in metadata_texts.index else ""
              for c in level0]
    level2 = []
    for c in level0:
        if c.startswith("ASQ_"):
            q = int(c.split("_",1)[1])
            level2.append("Reversed" if q in macro_2_questions else "Normal")
        else:
            level2.append("")

    scored_data.columns = pd.MultiIndex.from_tuples(
        list(zip(level0, level1, level2)),
        names=["Variable","Label","Scoring"]
    )

    # save and return
    scored_data.to_csv(path, index=False)
    return scored_data

## BIS

In [ ]:
# Barrett Impulsivity Scale
def score_bis(df, path):

    # Mapping of text answers to scores
    answer_scores = {
        "Rarely/Never": 1,
        "Occasionally": 2,
        "Often": 3,
        "Almost always/Always": 4
    }

    # Function to clean and map scores
    def map_scores(value):
        # Clean the string to remove leading/trailing whitespaces
        value = str(value).strip()
        # Return the mapped score or NaN if the value is not found
        return answer_scores.get(value, np.nan)

    # Function to reverse score the BIS items
    def BIS_reverse_score(value):
        if pd.isna(value):
            return value  # Preserve NaN values
        return 5 - value

    # Other commands to score the BIS survey
    bis_columns = [f'BIS_{i}' for i in range(1, 31)]
    n_items = len(bis_columns)

    scored_data = df[bis_columns].copy()

    # Apply mapping and cleaning
    scored_data = scored_data.applymap(map_scores)

    # Apply reverse scoring
    reverse_scored_items = [1, 7, 8, 9, 10, 12, 13, 15, 20, 29, 30]
    reverse_columns = [f'BIS_{item}' for item in reverse_scored_items]
    scored_data[reverse_columns] = scored_data[reverse_columns].applymap(BIS_reverse_score)


    # Count blanks & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Subscale item lists
    attentional_items   = ['BIS_5','BIS_6','BIS_9','BIS_11','BIS_20','BIS_24','BIS_26','BIS_28']
    motor_items         = ['BIS_2','BIS_3','BIS_4','BIS_16','BIS_17','BIS_19','BIS_21','BIS_22','BIS_23','BIS_25','BIS_30']
    nonplanning_items   = ['BIS_2','BIS_7','BIS_8','BIS_10','BIS_12','BIS_13','BIS_14','BIS_15','BIS_18','BIS_27','BIS_29']

    # Conditional-sum helper
    def cond_sum(row, items):
        return row[items].sum() if row['Blank_Count'] <= n_items/2 else np.nan

    # Compute subscales only if ≥ half answered
    scored_data['BIS_Attentional_Scores'] = scored_data.apply(lambda r: cond_sum(r, attentional_items), axis=1)
    scored_data['BIS_Motor_Scores']       = scored_data.apply(lambda r: cond_sum(r, motor_items), axis=1)
    scored_data['BIS_Nonplanning_Scores'] = scored_data.apply(lambda r: cond_sum(r, nonplanning_items), axis=1)

    # Conditional final, SD, and questions-not-answered
    scored_data['BIS_Final_Score'] = scored_data.apply(
        lambda r: r[bis_columns].sum(min_count=1) if r['Blank_Count'] <= n_items/2 else np.nan,
        axis=1
    )
    scored_data['BIS_SD_of_Answers'] = scored_data.apply(
        lambda r: r[bis_columns].std() if r['Blank_Count'] <= n_items/2 else np.nan,
        axis=1
    )
    scored_data['BIS_Questions_Not_Answered'] = scored_data['Blank_Count']

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # Add metadata as a multi-index
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)

    # Save to CSV
    output_data.to_csv(path, index=False)

    return output_data

## Big Five Inventory

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Big Five Inventory (BFI) scoring
def score_bfi(df, path):
    # Mapping of text answers to scores
    answer_scores = {
        "Disagree strongly": 1,
        "Disagree a little": 2,
        "Neither agree nor disagree": 3,
        "Agree a little": 4,
        "Agree strongly": 5
    }

    # Function to clean and map scores
    def map_scores(value):
        # Clean the string to remove leading/trailing whitespaces
        value = str(value).strip()
        # Return the mapped score or NaN if the value is not found
        return answer_scores.get(value, np.nan)

    # Function to reverse score the BFI items
    def BFI_reverse_score(value):
        if pd.isna(value):
            return value  # Preserve NaN values
        return 6 - value

    # List of BFI items
    bfi_columns = [f'BFI_{i}' for i in range(1, 45)]  # Adjust the range according to the number of items in BFI
    n_items = len(bfi_columns)


    # Create a copy of the relevant columns
    scored_data = df[bfi_columns].copy()

    # Apply mapping and cleaning
    scored_data = scored_data.applymap(map_scores)

    # Apply reverse scoring
    #reverse_scored_items = [1, 6, 11, 16, 21, 26, 31, 36]  # Adjust the list according to BFI reverse-scored items
    reverse_scored_items = [6, 21, 31, 2,12,27,37,8,18,23,43,9,24,34,35,41]
    reverse_columns = [f'BFI_{item}' for item in reverse_scored_items]
    scored_data[reverse_columns] = scored_data[reverse_columns].applymap(BFI_reverse_score)
    scored_data_columns = scored_data.copy()


    # Blank count & percentage
    scored_data['Blank_Count']      = scored_data[bfi_columns].isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Define your trait‐to‐item mappings
    traits = {
        'BFI_Extraversion': [1, 6, 11, 16, 21, 26, 31, 36],
        'BFI_Agreeableness': [2, 7, 12, 17, 22, 27, 32, 37, 42],
        'BFI_Conscientiousness': [3, 8, 13, 18, 23, 28, 33, 38, 43],
        'BFI_Neuroticism': [4, 9, 14, 19, 24, 29, 34, 39],
        'BFI_Openness': [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]
    }

    # Conditional subscale scoring (only if ≥ half answered)
    for trait, indices in traits.items():
        cols = [f'BFI_{i}' for i in indices]
        scored_data[trait] = scored_data.apply(
            lambda row: row[cols].mean()
                        if row['Blank_Count'] <= n_items/2
                        else np.nan,
            axis=1
        )

    # Conditional final and SD scores using the original cleaned item-only DataFrame
    scored_data['BFI_Final_Score'] = scored_data.apply(
        lambda row: scored_data_columns.loc[row.name, bfi_columns].mean()
                    if row['Blank_Count'] <= n_items / 2
                    else np.nan,
        axis=1
    )

    scored_data['BFI_SD_of_Answers'] = scored_data.apply(
        lambda row: scored_data_columns.loc[row.name, bfi_columns].std()
                    if row['Blank_Count'] <= n_items / 2
                    else np.nan,
        axis=1
    )

    scored_data['BFI_Questions_Not_Answered'] = scored_data['Blank_Count']


    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # Add metadata as a multi-index
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)

    # Save to CSV
    output_data.to_csv(path, index=False)
    return output_data


## ADHD

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# ADHD survey scoring
def score_adhd(df, path):

    # Numeric mapping (no entry for “Prefer not to respond” here)
    answer_scores = {
        "Not at all": 0,
        "Just a little": 1,
        "Somewhat": 2,
        "Moderately": 3,
        "Quite a lot": 4,
        "Very Much": 5
    }

    # Mapping fn for numeric responses
    def map_scores(value):
        return answer_scores.get(str(value).strip(), np.nan)

    # Column list + count
    adhd_columns = [f'ADHD_{i}' for i in range(1, 24)]
    n_items = len(adhd_columns)

    # Raw text, then map numeric
    scored_data = df[adhd_columns].copy()
    scored_data = scored_data.applymap(map_scores)

    # Put “Prefer not to respond” back in for anyone who chose it
    for col in adhd_columns:
        mask = df[col].astype(str).str.strip() == "Prefer not to respond"
        scored_data.loc[mask, col] = "Prefer not to respond"

    # Count blanks (only true NaNs) and percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Compute total, coercing non-numeric back to NaN for the sum
    def compute_total(row):
        if row['Blank_Count'] <= n_items/2:
            vals = pd.to_numeric(row[adhd_columns], errors='coerce')
            return vals.sum(min_count=1)
        else:
            return np.nan

    scored_data['ADHD_Total_Score'] = scored_data.apply(compute_total, axis=1)

    # Compute SD of the numeric items
    #    (non-numerics become NaN via to_numeric)
    scored_data['ADHD_Item_SD'] = scored_data[adhd_columns]\
        .apply(lambda row: pd.to_numeric(row, errors='coerce').std(skipna=True), axis=1)

    # Add hashed PROLIFIC_PID
    scored_data.insert(
        0,
        'Subject_ID',
        df[s1_prolific_id].apply(hash_id)
    )

    # Add metadata as a multi-index and save
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)
    output_data.to_csv(path, index=False)

    return output_data

## STAI

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive


# STAI scoring
def score_stai(df, path):
    # Mapping of text answers to scores for the state scale (questions 1-20)
    state_answer_scores = {
        "Not At All": 1,
        "Somewhat": 2,
        "Moderately So": 3,
        "Very Much So": 4
    }

    # Mapping of text answers to scores for the trait scale (questions 21-40)
    trait_answer_scores = {
        "Almost Never": 1,
        "Sometimes": 2,
        "Often": 3,
        "Almost Always": 4
    }

    # Function to clean and map scores for state questions
    def map_state_scores(value):
        value = str(value).strip()
        return state_answer_scores.get(value, np.nan)

    # Function to clean and map scores for trait questions
    def map_trait_scores(value):
        value = str(value).strip()
        return trait_answer_scores.get(value, np.nan)

    # List of STAI items
    state_columns = [f'STAI_1_{i}' for i in range(1, 21)]
    trait_columns = [f'STAI_2_{i}' for i in range(1, 21)]
    n_state = len(state_columns)
    n_trait = len(trait_columns)

    # Create a copy of the relevant columns
    state_scored_data = df[state_columns].copy()
    trait_scored_data = df[trait_columns].copy()

    # Apply mapping and cleaning
    state_scored_data = state_scored_data.applymap(map_state_scores)
    trait_scored_data = trait_scored_data.applymap(map_trait_scores)

    # Count blanks & percentage
    state_scored_data["STAI_State_Blank_Count"]      = state_scored_data.isna().sum(axis=1)
    state_scored_data["STAI_State_Blank_Percentage"] = state_scored_data["STAI_State_Blank_Count"] / n_state
    trait_scored_data["STAI_Trait_Blank_Count"]      = trait_scored_data.isna().sum(axis=1)
    trait_scored_data["STAI_Trait_Blank_Percentage"] = trait_scored_data["STAI_Trait_Blank_Count"] / n_trait

    # Conditional mean & SD functions
    def cond_mean(row, items, blank_field, n):
        if row[blank_field] <= n/2:
            return row[items].mean()
        return np.nan

    def cond_sd(row, items, blank_field, n):
        if row[blank_field] <= n/2:
            return row[items].std()
        return np.nan

    # 7) Compute State & Trait metrics only when ≥ half answered
    state_scored_data["STAI_State_Mean"] = state_scored_data.apply(
        lambda r: cond_mean(r, state_columns, "STAI_State_Blank_Count", n_state),
        axis=1
    )
    state_scored_data["STAI_State_SD"] = state_scored_data.apply(
        lambda r: cond_sd(r, state_columns, "STAI_State_Blank_Count", n_state),
        axis=1
    )
    trait_scored_data["STAI_Trait_Mean"] = trait_scored_data.apply(
        lambda r: cond_mean(r, trait_columns, "STAI_Trait_Blank_Count", n_trait),
        axis=1
    )
    trait_scored_data["STAI_Trait_SD"] = trait_scored_data.apply(
        lambda r: cond_sd(r, trait_columns, "STAI_Trait_Blank_Count", n_trait),
        axis=1
    )

    # 8) Combine and total questions skipped
    scored_data = pd.concat([state_scored_data, trait_scored_data], axis=1)
    scored_data["STAI_Questions_Not_Answered"] = (
        scored_data["STAI_State_Blank_Count"] +
        scored_data["STAI_Trait_Blank_Count"]
    )

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # Add metadata as a multi-index
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)
    output_data.to_csv(path, index=False)
    return output_data

## OCD

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive
import re

# OCD survey scoring
def score_ocd(df, path):

    # Mapping of text answers to 0–4
    answer_scores_1_5 = {
        "None at all": 0,
        "Less than 1 hour each day": 1,
        "Between 1 and 3 hours each day": 2,
        "Between 3 and 8 hours each day": 3,
        "8 hours or more each day": 4,

        "A little avoidance": 1,
        "A moderate amount of avoidance": 2,
        "A great deal of avoidance": 3,
        "Extreme avoidance of nearly all things": 4,

        "Not at all distressed/anxious": 0,
        "Mildly distressed/anxious": 1,
        "Moderately distressed/anxious": 2,
        "Severely distressed/anxious": 3,
        "Extremely distressed/anxious": 4,

        "No disruption at all.": 0,
        "A little disruption, but I mostly function well.": 1,
        "Many things are disrupted, but I can still manage.": 2,
        "My life is disrupted in many ways and I have trouble managing.": 3,
        "My life is completely disrupted and I cannot function at all.": 4,

        "Not at all difficult": 0,
        "A little difficult": 1,
        "Moderately difficult": 2,
        "Very difficult": 3,
        "Extremely difficult": 4
    }

    def map_scores(value):
        return answer_scores_1_5.get(str(value).strip(), np.nan)

    # Define items & count
    ocd_columns = [f'OCD_{i}_{j}' for i in range(1, 5) for j in range(1, 6)]
    n_items     = len(ocd_columns)

    # Score items
    scored_data = df[ocd_columns].copy().applymap(map_scores)

    # Blank counts & percentages
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Total score (only if ≥ half answered)
    def cond_sum(row, cols):
        return row[cols].sum() if row['Blank_Count'] <= n_items/2 else np.nan

    scored_data['OCD_Total_Score'] = scored_data.apply(lambda r: cond_sum(r, ocd_columns), axis=1)

    # Subscale scores
    subs = {
      'OCD_1_5_Score':   [f'OCD_1_{i}' for i in range(1, 6)],
      'OCD_6_10_Score':  [f'OCD_2_{i}' for i in range(1, 6)],
      'OCD_11_15_Score': [f'OCD_3_{i}' for i in range(1, 6)],
      'OCD_16_20_Score': [f'OCD_4_{i}' for i in range(1, 6)],
    }
    for name, cols in subs.items():
        scored_data[name] = scored_data.apply(lambda r: cond_sum(r, cols), axis=1)

    # Standard deviation of the numeric items
    scored_data['OCD_Item_SD'] = scored_data[ocd_columns]\
        .std(axis=1, skipna=True)

    # Hash ID & metadata, then save
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))
    output = add_metadata_as_multiindex(scored_data, s1_metadata_row)

    # Mel: Add Subscale scores
    # Extract the main/outer level index
    main_level = scored_data.columns.get_level_values(0)

    main_level = (
        main_level
        .str.replace(r'^OCD_1_(\d+)', r'OCD_germs_\1', regex=True)
        .str.replace(r'^OCD_2_(\d+)', r'OCD_harm_\1', regex=True)
        .str.replace(r'^OCD_3_(\d+)', r'OCD_thoughts_\1', regex=True)
        .str.replace(r'^OCD_4_(\d+)', r'OCD_symmetry_\1', regex=True)
    )
    # Reassign back to the MultiIndex
    scored_data.columns = pd.MultiIndex.from_arrays([main_level] + [scored_data.columns.get_level_values(i) for i in range(1, scored_data.columns.nlevels)])

    output.to_csv(path, index=False)
    return output

## Grit


In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Grit Scale survey scoring
def score_grit(df, path):
    # Mapping of text answers to 1–5
    answer_scores = {
        "Not like me at all": 1,
        "Not much like me": 2,
        "Somewhat like me": 3,
        "Mostly like me": 4,
        "Very much like me": 5
    }

    def map_scores(value):
        return answer_scores.get(str(value).strip(), np.nan)

    # Item list & count
    grit_columns = [
        'Grit Scale_1', 'Grit Scale_6', 'Grit Scale_2', 'Grit Scale_3',
        'Grit Scale_4', 'Grit Scale_7', 'Grit Scale_5', 'Grit Scale_8',
        'Grit Scale_9', 'Grit Scale_10'
    ]
    n_items = len(grit_columns)

    # Score items
    scored_data = df[grit_columns].copy().applymap(map_scores)

    # Blank counts & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Total score (only if ≥ half answered)
    scored_data['Grit_Total_Score'] = scored_data.apply(
        lambda row: row[grit_columns].sum(min_count=1)
                    if row['Blank_Count'] <= n_items/2
                    else np.nan,
        axis=1
    )

    # Item‐wise standard deviation
    scored_data['Grit_Item_SD'] = scored_data[grit_columns]\
        .std(axis=1, skipna=True)

    # Hash ID, add metadata, save
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)
    output_data.to_csv(path, index=False)

    return output_data

In [ ]:
def save_with_metadata(df, path):
    # Extract header rows from the MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        headers = pd.DataFrame(df.columns.tolist()).T
        df.columns = [''] * df.shape[1]  # temporary dummy columns for correct format
        with open(path, 'w', encoding='utf-8', newline='') as f:
            headers.to_csv(f, index=False, header=False)
            df.to_csv(f, index=False, header=False)
    else:
        df.to_csv(path, index=False)

## Oldenburg Burnout Inventory


In [ ]:
import pandas as pd
import numpy as np
import hashlib

def score_oldenburg(df, path):
    # Items 1–16
    oldenburg_columns = [f'Burnout_{i}' for i in range(1, 17)]
    n_items = len(oldenburg_columns)

    # Reverse‐scoring items
    reverse_items = {2, 3, 4, 6, 8, 9, 11, 12}

    # Map textual responses to 1–4
    response_mapping = {
        'strongly agree': 1,
        'agree': 2,
        'disagree': 3,
        'strongly disagree': 4
    }
    scored = df[oldenburg_columns].replace(response_mapping)

    # Reverse‐score designated items (1 <-> 4, 2 <-> 3)
    def _rev(x):
        if pd.isna(x):
            return np.nan
        return 5 - x  # flips 1→4, 2→3, 3→2, 4→1

    rev_cols = [f'Burnout_{i}' for i in reverse_items]
    scored[rev_cols] = scored[rev_cols].applymap(_rev)

    # Compute blanks & percentage
    scored['Blank_Count']      = scored.isna().sum(axis=1)
    scored['Blank_Percentage'] = scored['Blank_Count'] / n_items

    # Subscales
    exhaustion_items   = [2, 4, 5, 8, 10, 12, 14, 16]
    disengagement_items = [1, 3, 6, 7, 9, 11, 13, 15]

    exhaustion_cols   = [f'Burnout_{i}' for i in exhaustion_items]
    disengagement_cols = [f'Burnout_{i}' for i in disengagement_items]

    scored['Exhaustion_Score']    = scored[exhaustion_cols].sum(axis=1, min_count=1)
    scored['Disengagement_Score'] = scored[disengagement_cols].sum(axis=1, min_count=1)

    # Total (only if ≥ half answered)
    scored['Oldenburg_Total_Score'] = scored.apply(
        lambda r: r[oldenburg_columns].sum(min_count=1)
                  if r['Blank_Count'] <= n_items/2 else np.nan,
        axis=1
    )

    # Item‐level SD
    scored['Oldenburg_Item_SD'] = scored[oldenburg_columns].std(axis=1, skipna=True)

    # hashed ID
    scored.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # ---- Add only question text + Normal/Reversed rows ----
    metadata_texts = s1_metadata_row  # use stored metadata row instead of raw answers
    level0 = scored.columns
    level1 = [metadata_texts.get(c, "") if c in metadata_texts.index else "" for c in level0]
    level2 = []
    for c in level0:
        if c.startswith("Burnout_"):
            q = int(c.split("_",1)[1])
            level2.append("Reversed" if q in reverse_items else "Normal")
        else:
            level2.append("")

    scored.columns = pd.MultiIndex.from_tuples(
        list(zip(level0, level1, level2)),
        names=["Variable","Question Text","Scoring"]
    )

    # Save
    scored.to_csv(path, index=False)
    return scored

## Maslach Burnout Inventory


In [ ]:
import pandas as pd
import numpy as np
import hashlib

def score_maslach(df, path):
    # Define MBI subscales
    exhaustion_items = [f'Maslach_{i}' for i in [1, 2, 3, 4, 6]]
    cynicism_items = [f'Maslach_{i}' for i in [8, 9, 13, 14, 15]]
    professional_efficacy_items = [f'Maslach_{i}' for i in [5, 7, 10, 11, 12, 16]]
    maslach_columns = exhaustion_items + cynicism_items + professional_efficacy_items
    n_items = len(maslach_columns)

    # Response mapping
    response_mapping = {
        'Never': 0,
        'A few times a year or less': 1,
        'Once a month or less': 2,
        'A few times a month': 3,
        'Once a week': 4,
        'A few times a week': 5,
        'Every Day': 6
    }

    # Track missing before transformation
    df = df.copy()
    df['Missing_Fields_Count_Original'] = df[maslach_columns].isna().sum(axis=1)

    # Map responses to numeric
    scored_data = df[maslach_columns].replace(response_mapping).apply(pd.to_numeric, errors='coerce')

    # Missing counts
    scored_data['Missing_Fields_Count_Transformed'] = scored_data.isna().sum(axis=1)
    scored_data['Blank_Count']      = scored_data['Missing_Fields_Count_Transformed']
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Conditional sums & means
    def conditional_sum(row, items):
        return row[items].sum(skipna=True) if row['Blank_Count'] <= n_items/2 else np.nan
    def conditional_mean(row, items):
        return row[items].mean(skipna=True) if row['Blank_Count'] <= n_items/2 else np.nan

    scored_data['Exhaustion_Total_Score']            = scored_data.apply(lambda r: conditional_sum(r, exhaustion_items), axis=1)
    scored_data['Cynicism_Total_Score']              = scored_data.apply(lambda r: conditional_sum(r, cynicism_items), axis=1)
    scored_data['Professional_Efficacy_Total_Score'] = scored_data.apply(lambda r: conditional_sum(r, professional_efficacy_items), axis=1)

    scored_data['Exhaustion_Average_Score']            = scored_data.apply(lambda r: conditional_mean(r, exhaustion_items), axis=1)
    scored_data['Cynicism_Average_Score']              = scored_data.apply(lambda r: conditional_mean(r, cynicism_items), axis=1)
    scored_data['Professional_Efficacy_Average_Score'] = scored_data.apply(lambda r: conditional_mean(r, professional_efficacy_items), axis=1)

    # SD across items
    scored_data['Maslach_Item_SD'] = scored_data[maslach_columns].std(axis=1, skipna=True)

    # --- Add reversed Professional Efficacy columns ---
    def reverse_score(x):
        if pd.isna(x): return np.nan
        return 6 - x  # flips 0→6, 1→5, 2→4, 3→3, 4→2, 5→1, 6→0

    reversed_prof = scored_data[professional_efficacy_items].map(reverse_score)
    scored_data['Professional_Efficacy_Total_Score_Reversed'] = reversed_prof.sum(axis=1, min_count=1)
    scored_data['Professional_Efficacy_Average_Score_Reversed'] = reversed_prof.mean(axis=1)

    # Add hashed Subject ID
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # Add metadata multi-index
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)

    # Save
    output_data.to_csv(path, index=False)
    return output_data

## BFNE

In [ ]:
# BFNE survey scoring
def score_bfne(df, path):
    # Map text answers to numeric scores
    answer_scores = {
        "Not at all characteristic or true of me": 1,
        "Slightly characteristic or true of me": 2,
        "Moderately characteristic or true of me": 3,
        "Very characteristic or true of me": 4,
        "Extremely characteristic or true of me": 5
    }

    def map_scores(value):
        return answer_scores.get(str(value).strip(), np.nan)

    # Define BFNE columns
    bfne_columns = [f'BFNE_{i}' for i in range(1, 9)]
    n_items = len(bfne_columns)

    # Score each item
    scored_data = df[bfne_columns].applymap(map_scores)

    # Count blanks & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Conditional total: only if at least half answered
    scored_data['BFNE_Total_Score'] = scored_data.apply(
        lambda row: row[bfne_columns].sum(min_count=1)
                    if row['Blank_Count'] <= n_items / 2
                    else np.nan,
        axis=1
    )

    # Standard deviation
    scored_data['BFNE_Item_SD'] = scored_data[bfne_columns].std(axis=1, skipna=True)

    # Add hashed ID and metadata, then save
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)
    output_data.to_csv(path, index=False)

    return output_data

## CUSADOS


In [ ]:
# CUSADOS survey scoring
def score_cusados(df, path):
    # Map text answers to numeric scores
    answer_scores = {
        "never": 0,
        "rarely": 1,
        "sometimes": 2,
        "usually": 3,
        "always": 4
    }

    def map_scores(value):
        return answer_scores.get(str(value).strip().lower(), np.nan)

    # Define your CUSADOS columns
    cusados_columns = [f'CUSADOS_{i}' for i in range(1, 13)]
    n_items = len(cusados_columns)

    # Score each item
    scored_data = df[cusados_columns].applymap(map_scores)

    # Count blanks & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Conditional total: only if at least half answered
    scored_data['CUSADOS_Total_Score'] = scored_data.apply(
        lambda row: row[cusados_columns].sum(min_count=1)
                    if row['Blank_Count'] <= n_items/2
                    else np.nan,
        axis=1
    )

    # Standard deviation across the 12 items
    scored_data['CUSADOS_Item_SD'] = scored_data[cusados_columns].std(axis=1, skipna=True)

    # Add hashed ID, metadata and save
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)
    output_data.to_csv(path, index=False)

    return output_data

## UCLA Loneliness

In [ ]:
# Loneliness survey scoring
def score_ucla_loneliness(df, path):
    # Map text answers to numeric scores
    answer_scores = {
        "hardly ever": 1,
        "some of the time": 2,
        "often": 3
    }

    def map_scores(value):
        return answer_scores.get(str(value).strip().lower(), np.nan)

    # Define UCLA Loneliness columns
    loneliness_columns = [f'UCLA_Loneliness_{i}' for i in range(1, 4)]
    n_items = len(loneliness_columns)

    # Score each item
    scored_data = df[loneliness_columns].applymap(map_scores)

    # Blank counts & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Conditional total: only if at least half answered
    scored_data['Loneliness_Total_Score'] = scored_data.apply(
        lambda row: row[loneliness_columns].sum(min_count=1)
                    if row['Blank_Count'] <= n_items/2
                    else np.nan,
        axis=1
    )

    # Standard deviation across the 3 items
    scored_data['UCLA_Loneliness_SD'] = scored_data[loneliness_columns].std(axis=1, skipna=True)

    # Add hashed ID, metadata, save
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)
    output_data.to_csv(path, index=False)

    return output_data

## Gallup Best Friend

In [ ]:
# Function to extract Gallup Best Friend Item response
def score_gallup_bff(df, path):
    # List of GallupBFF item
    gallup_bff_column = 'GallupBFF_1'

    # Create a copy of the relevant column
    scored_data = df[[gallup_bff_column]].copy()

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))

    # Add metadata as a multi-index
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)

    # Save to CSV
    output_data.to_csv(path, index=False)

    return output_data

  # some of these items are numeric some are 'strongly disagree' etc.

## AUDIT


In [ ]:
# Function to score the AUDIT survey
def score_audit(df, path):
    import numpy as np
    import pandas as pd

    # Map text to values
    answer_scores = {
        "never":                  0,
        "monthly or less":        1,
        "less than monthly":      1,
        "2-4 times a month":      2,
        "monthly":                2,
        "2-3 times a week":       3,
        "weekly":                 3,
        "4 or more times a week": 4,
        "daily or almost daily":  4,

        "1 or 2":                 0,
        "3 or 4":                 1,
        "5 or 6":                 2,
        "7 to 9":                 3,
        "10 or more":             4,

        "no":                     0,
        "yes, but not in the last year": 2,
        "yes, during the last year":    4
    }

    def map_scores(value):
        return answer_scores.get(str(value).strip().lower(), np.nan)

    # Define AUDIT columns
    audit_columns = [f'AUDIT_{i}' for i in range(1, 11)]
    n_items = len(audit_columns)

    # Score each item
    scored_data = df[audit_columns].copy().applymap(map_scores)

    # Subscales
    scored_data['AUDIT_Consumption_Score'] = scored_data[['AUDIT_1','AUDIT_2','AUDIT_3']].sum(axis=1, min_count=1)
    scored_data['AUDIT_Problem_Score']     = scored_data[['AUDIT_4','AUDIT_5','AUDIT_6','AUDIT_7','AUDIT_8','AUDIT_9','AUDIT_10']].sum(axis=1, min_count=1)

    # Blank counts & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Conditional total (only if ≥ half answered)
    scored_data['AUDIT_Total_Score'] = scored_data.apply(
        lambda row: row[audit_columns].sum(min_count=1)
                    if row['Blank_Count'] <= n_items/2
                    else np.nan,
        axis=1
    )

    # Standard deviation across the 10 items
    scored_data['AUDIT_Item_SD'] = scored_data[audit_columns].std(axis=1, skipna=True)

    # Hashed ID, metadata, save
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)
    output_data.to_csv(path, index=False)

    return output_data

## BRCS

In [ ]:
# Function to score the BRCS survey
def score_brcs(df, path):
    import numpy as np
    import pandas as pd

    # Mapping of text answers to scores
    answer_scores = {
        "does not describe me at all": 1,
        "does not describe me": 2,
        "neutral": 3,
        "describes me": 4,
        "describes me very well": 5
    }

    # Clean-and-map function
    def map_scores(value):
        return answer_scores.get(str(value).strip().lower(), np.nan)

    # Define 4 BRCS columns
    brcs_columns = [f'BRCS_{i}' for i in range(1, 5)]
    n_items = len(brcs_columns)

    # Score each item
    scored_data = df[brcs_columns].copy().applymap(map_scores)

    # Blank counts & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Conditional total (only if ≥ half answered)
    scored_data['BRCS_Total_Score'] = scored_data.apply(
        lambda row: row[brcs_columns].sum(min_count=1)
                    if row['Blank_Count'] <= n_items/2
                    else np.nan,
        axis=1
    )

    # Interpretation
    def categorize_score(score):
        if pd.isna(score):
            return np.nan
        if score <= 13:
            return "Low resilient coper"
        elif score <= 16:
            return "Medium resilient coper"
        else:
            return "High resilient coper"

    scored_data['BRCS_Interpretation'] = scored_data['BRCS_Total_Score'].apply(categorize_score)

    # SD across the 4 items
    scored_data['BRCS_Item_SD'] = scored_data[brcs_columns].std(axis=1, skipna=True)

    # Hashed ID, metadata, save
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)
    output_data.to_csv(path, index=False)

    return output_data

## Pastimes


In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Function to score the Pastimes survey
def score_pastimes(df, path):
    # Define columns
    pastimes_columns = [f'Pastimes_{i}' for i in range(1, 19)]
    nonnum = pastimes_columns[0]      # e.g. 'By Myself' / 'With Others'
    num_cols = pastimes_columns[1:]   # numeric‐scored items
    n_numeric = len(num_cols)

    # Text to numeric mapping
    answer_scores = {
        "never": 0,
        "1-3 hours": 3,
        "4-6 hours": 6,
        "7-9 hours": 9,
        "10+ hours": 10
    }
    def map_scores(value):
        return answer_scores.get(str(value).strip().lower(), np.nan)

    # Copy & map
    scored_data = df[pastimes_columns].copy()
    scored_data[num_cols] = (
        scored_data[num_cols]
        .applymap(map_scores)
        .apply(pd.to_numeric, errors='coerce')
    )

    # Blank counts & percentage (numeric only)
    scored_data['Blank_Count']      = scored_data[num_cols].isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_numeric

    # Conditional total on numeric only
    scored_data['Pastimes_Total_Score'] = scored_data.apply(
        lambda r: r[num_cols].sum(min_count=1)
                  if r['Blank_Count'] <= n_numeric/2
                  else np.nan,
        axis=1
    )

    # SD across the numeric items
    scored_data['Pastimes_Item_SD'] = scored_data[num_cols].std(axis=1, skipna=True)

    # Hashed ID + metadata + save
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)
    output_data.to_csv(path, index=False)

    return output_data

## PSQI

In [ ]:
def score_psqi(df, path):
    import numpy as np
    import pandas as pd

    # Define the PSQI columns to process
    psqi_columns = [
        "PSQI_1_1", "PSQI_2_1", "PSQI_3_1", "PSQI_4_1", "PSQI_tib_1",
        "PSQI_5a", "PSQI_5b", "PSQI_5c", "PSQI_5d", "PSQI_5e",
        "PSQI_5f", "PSQI_5g", "PSQI_5h", "PSQI_5i", "PSQI_5j_2",
        "PSQI_6", "PSQI_7", "PSQI_8", "PSQI_9"
    ]
    n_items = len(psqi_columns)

    # Create a working copy of the PSQI data
    scored_data = df[psqi_columns].copy()

    # === Clean Numeric Columns ===
    for col in ["PSQI_2_1", "PSQI_4_1", "PSQI_tib_1"]:
        scored_data[col] = scored_data[col].replace("", np.nan)
        scored_data[col] = scored_data[col].astype(str).str.extract(r"(\d+\.?\d*)")[0]
        scored_data[col] = pd.to_numeric(scored_data[col], errors="coerce")

    # === Mapping Dictionaries ===
    sleep_freq_map = {
        "Not during the past month": 0,
        "Not during the last month": 0,
        "Less than once a week": 1,
        "Once or twice a week": 2,
        "Three or more times a week": 3,
        "N/A": 0
    }
    sleep_quality_map = {
        "Very good": 0,
        "Fairly good": 1,
        "Fairly bad": 2,
        "Very bad": 3
    }

    # === Apply Mapping to Frequency Items ===
    freq_cols = ["PSQI_5a","PSQI_5b","PSQI_5c","PSQI_5d","PSQI_5e",
                 "PSQI_5f","PSQI_5g","PSQI_5h","PSQI_5i","PSQI_5j_2","PSQI_8"]
    scored_data[freq_cols] = scored_data[freq_cols].apply(
        lambda col: col.astype(str).str.strip().map(sleep_freq_map)
    )
    scored_data["PSQI_9"] = scored_data["PSQI_9"].astype(str).str.strip().map(sleep_freq_map)

    # === Sleep Duration Score (PSQIDURAT) ===
    def compute_psqidurat(value):
        if pd.isna(value): return np.nan
        if value >= 7:                       return 0
        elif 6 <= value < 7:                 return 1
        elif 5 <= value < 6:                 return 2
        elif value < 5:                      return 3
        else:                                return np.nan
    scored_data["PSQIDURAT"] = scored_data["PSQI_4_1"].apply(compute_psqidurat)

    # === Sleep Latency Scoring (PSQILATEN) ===
    def compute_q2new(value):
        if pd.isna(value): return np.nan
        if 0 <= value <= 15:         return 0
        elif 15 < value <= 30:       return 1
        elif 30 < value <= 60:       return 2
        elif value > 60:             return 3
        else:                         return np.nan
    scored_data["Q2new"] = scored_data["PSQI_2_1"].apply(compute_q2new)

    def compute_psqilaten(row):
        if pd.isna(row["PSQI_5a"]) or pd.isna(row["Q2new"]):
            return np.nan
        total_latency = row["PSQI_5a"] + row["Q2new"]
        if total_latency == 0:        return 0
        elif 1 <= total_latency <= 2:  return 1
        elif 3 <= total_latency <= 4:  return 2
        elif 5 <= total_latency <= 6:  return 3
        else:                          return np.nan
    scored_data["PSQILATEN"] = scored_data.apply(compute_psqilaten, axis=1)

    # === Sleep Efficiency Scoring (PSQIHSE) ===
    def compute_psqihse(row):
        if pd.isna(row["PSQI_4_1"]) or pd.isna(row["PSQI_tib_1"]) or row["PSQI_tib_1"] == 0:
            return np.nan
        tmphse = (row["PSQI_4_1"] / row["PSQI_tib_1"]) * 100
        if tmphse >= 85:               return 0
        elif 75 <= tmphse < 85:        return 1
        elif 65 <= tmphse < 75:        return 2
        elif tmphse < 65:              return 3
        else:                          return np.nan
    scored_data["PSQIHSE"] = scored_data.apply(compute_psqihse, axis=1)

    # === Sleep Disturbance Scoring (PSQIDISTB) ===
    disturbance_cols = ["PSQI_5b","PSQI_5c","PSQI_5d","PSQI_5e","PSQI_5f",
                        "PSQI_5g","PSQI_5h","PSQI_5i","PSQI_5j_2"]
    scored_data["PSQI_5j_2"].fillna(0, inplace=True)
    scored_data["PSQIDISTB_raw"] = scored_data[disturbance_cols].sum(axis=1, min_count=1)
    def compute_psqidistb(row):
        td = row["PSQIDISTB_raw"]
        if td == 0:                   return 0
        elif 1 <= td <= 9:            return 1
        elif 10 <= td <= 18:          return 2
        elif td > 18:                 return 3
        else:                         return np.nan
    scored_data["PSQIDISTB"] = scored_data.apply(compute_psqidistb, axis=1)

    # === Daytime Dysfunction Scoring (PSQIDAYDYS) ===
    scored_data[["PSQI_8","PSQI_9"]] = scored_data[["PSQI_8","PSQI_9"]].fillna(0)
    scored_data["PSQIDAYDYS_raw"] = scored_data["PSQI_8"] + scored_data["PSQI_9"]
    def compute_psqidaydys(total):
        if total == 0:                return 0
        elif 1 <= total <= 2:         return 1
        elif 3 <= total <= 4:         return 2
        elif 5 <= total <= 6:         return 3
        else:                         return np.nan
    scored_data["PSQIDAYDYS"] = scored_data["PSQIDAYDYS_raw"].apply(compute_psqidaydys)

    # === Overall Sleep Quality & Medication ===
    scored_data["PSQISLPQUAL"] = scored_data["PSQI_6"].astype(str).str.strip().map(sleep_quality_map)
    scored_data["PSQIMEDS"]   = (scored_data["PSQI_7"]
                                 .astype(str)
                                 .str.strip()
                                 .replace("Not during the last month","Not during the past month")
                                 .map(sleep_freq_map))
    scored_data["PSQI_6"] = scored_data["PSQISLPQUAL"]
    scored_data["PSQI_7"] = scored_data["PSQIMEDS"]

    # === Compute Total PSQI Score ===
    scored_data['Blank_Count']      = scored_data[psqi_columns].isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    components = ["PSQIDURAT","PSQILATEN","PSQIHSE","PSQIDISTB","PSQIDAYDYS","PSQISLPQUAL","PSQIMEDS"]
    scored_data["PSQI_Total"] = scored_data.apply(
        lambda r: r[components].sum(min_count=1)
                  if r['Blank_Count'] <= n_items/2
                  else np.nan,
        axis=1
    )

    # === Add Sleep Quality Category ===
    scored_data["PSQI_Category"] = np.where(
        scored_data["PSQI_Total"] < 5, "Good Sleep Quality", "Poor Sleep Quality"
    )

    # === Standard Deviation across the 7 component scores ===
    scored_data["PSQI_Item_SD"] = scored_data[components].std(axis=1, skipna=True)

    # === Add Hashed Subject ID & Metadata, then Save ===
    scored_data.insert(0, 'Subject_ID', df['Q1'].apply(hash_id))
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)
    output_data.to_csv(path, index=False)

    return output_data

## Media Multi-Tasking Inventory (short) (MMTI-S)

In [ ]:
def score_mmti_s(df, path):
    import pandas as pd
    import numpy as np

    # List of MMTI-S items
    mmti_s_columns = [f'MMTI-S_{i}' for i in range(1, 10)]
    n_items = len(mmti_s_columns)

    # Pull raw and convert to numeric (coerce non-numeric → NaN)
    scored_data = df[mmti_s_columns].copy().apply(pd.to_numeric, errors='coerce')

    # Count blanks & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Conditional total: only if at least half answered
    scored_data['MMTI-S_Total_Score'] = scored_data.apply(
        lambda row: row[mmti_s_columns].sum(min_count=1)
                    if row['Blank_Count'] <= n_items/2
                    else np.nan,
        axis=1
    )

    # Conditional average
    scored_data['MMTI_S_Average_Score'] = scored_data.apply(
        lambda row: row[mmti_s_columns].mean(skipna=True)
                    if row['Blank_Count'] <= n_items/2
                    else np.nan,
        axis=1
    )

    # Standard deviation across the 9 items
    scored_data['MMTI_S_Item_SD'] = scored_data[mmti_s_columns].std(axis=1, skipna=True)

    # Add hashed `PROLIFIC_PID`
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))

    # Add metadata as a multi-index
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)

    # Save to CSV
    output_data.to_csv(path, index=False)

    return output_data

## PANAS

In [ ]:
# Function to score the PANAS (Positive and Negative Affect Schedule)
def score_panas(df, path):
    import numpy as np
    import pandas as pd

    # List of PANAS items
    panas_columns = [f'PANAS_{i}' for i in range(1, 21)]
    n_items = len(panas_columns)

    # Mapping of text answers to scores
    answer_scores = {
        "very slightly or not at all": 1,
        "a little": 2,
        "moderately": 3,
        "quite a bit": 4,
        "extremely": 5
    }

    # Function to map and clean scores
    def map_scores(value):
        return answer_scores.get(str(value).strip().lower(), np.nan)

    # Apply mapping
    scored_data = df[panas_columns].applymap(map_scores)

    # Count blanks & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Define positive and negative items
    positive_items = [1,3,5,9,10,12,14,16,17,19]
    negative_items = [2,4,6,7,8,11,13,15,18,20]

    # Compute subscale totals (only if at least half answered)
    scored_data['PANAS_Positive_Score'] = scored_data.apply(
        lambda row: row[[f'PANAS_{i}' for i in positive_items]].sum(min_count=1)
                    if row['Blank_Count'] <= n_items/2
                    else np.nan,
        axis=1
    )
    scored_data['PANAS_Negative_Score'] = scored_data.apply(
        lambda row: row[[f'PANAS_{i}' for i in negative_items]].sum(min_count=1)
                    if row['Blank_Count'] <= n_items/2
                    else np.nan,
        axis=1
    )

    # Standard deviation across all 20 items
    scored_data['PANAS_Item_SD'] = scored_data[panas_columns].std(axis=1, skipna=True)

    # Add hashed `PROLIFIC_PID`
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))

    # Add metadata as a multi-index
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)

    # Save to CSV
    output_data.to_csv(path, index=False)

    return output_data

## PFI


In [ ]:
# Function to score the Professional Fulfillment Index (PFI)
def score_pfi(df, path):
    import numpy as np
    import pandas as pd

    # List of PFI items
    pfi_columns = [f'PFI_{i}' for i in range(1, 17)]
    n_items = len(pfi_columns)

    # Mapping of text answers to scores
    answer_scores = {
        "Not at all": 0,
        "Very Little": 1,
        "Moderately": 2,
        "A lot": 3,
        "Extremely": 4
    }

    # Function to map and clean scores
    def map_scores(value):
        return answer_scores.get(str(value).strip(), np.nan)

    # Map all PFI items
    scored_data = df[pfi_columns].applymap(map_scores)

    # Count blanks & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Conditional total: only if at least half answered
    scored_data['PFI_Total_Score'] = scored_data.apply(
        lambda row: row[pfi_columns].sum(min_count=1)
                    if row['Blank_Count'] <= n_items/2
                    else np.nan,
        axis=1
    )

    # Calculate average score across all items (NaNs will be ignored)
    scored_data['PFI_Average_Score'] = scored_data[pfi_columns].mean(axis=1, skipna=True)

    # Standard deviation across all 16 items
    scored_data['PFI_Item_SD'] = scored_data[pfi_columns].std(axis=1, skipna=True)

    # Add hashed `PROLIFIC_PID`
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))

    # Add metadata as a multi-index
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)

    # Save to CSV
    output_data.to_csv(path, index=False)

    return output_data

## Wordle

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Big Five Inventory (BFI) scoring
def score_wordle(df, path):

    # List of Wordle items
    import re

    # Only match columns like 'Wordle_1', 'Wordle_2', etc. — no extra text after the number
    Wordle_columns = [col for col in df.columns if re.fullmatch(r'Wordle_\d+', col)]

    # Create a copy of the relevant columns
    scored_data = df[Wordle_columns].copy()

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # Add metadata as a multi-index (if applicable)
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)

    # Save the scored data
    scored_data.to_csv(path, index=False)

    return scored_data

## Cudit

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Big Five Inventory (BFI) scoring
def score_cudit(df, path):

  # Map text to values
    answer_scores = {
        "never":                  0,
        "monthly or less":        1,
        "2-4 times a month":      2,
        "2-3 times a week":       3,
        "4 or more times a week": 4,

        "less than 1":            0,
        "1 or 2":                 1,
        "3 or 4":                 2,
        "5 or 6":                 3,
        "7 or more":              4,

        "no":                     0,
        "yes":                    4,

        "never":                  0,
        "less than monthly":      1,
        "monthly":                2,
        "weekly":                 3,
        "daily or almost daily":  4
    }

    # Function to clean and map scores
    def map_scores(value):
        value = str(value).strip().lower()
        return answer_scores.get(value, np.nan)

    # List of Demographic items
    cudit_columns = [col for col in df.columns if col.startswith(f'CUDIT_')]
    n_items = len(cudit_columns)


    # Create a copy of the relevant columns
    scored_data = df[cudit_columns].copy()

    # Apply mapping and cleaning
    scored_data = scored_data.applymap(map_scores)

    # Count blanks & percentage
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Conditional total: only if at least half answered
    scored_data['CUDIT_Total_Score'] = scored_data.apply(
        lambda row: row[cudit_columns].sum()
                    if row['Blank_Count'] <= n_items / 2
                    else np.nan,
        axis=1
    )

    # Standard deviation of the numeric items
    scored_data['CUDIT_Item_SD'] = scored_data[cudit_columns]\
        .std(axis=1, skipna=True)


    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))

    # Add metadata as a multi-index (if applicable)
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)

    # Save the scored data
    scored_data.to_csv(path,index=False)

    return scored_data

## Demographics S1

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Big Five Inventory (BFI) scoring
def score_demographics(df, path):

    # List of Demographic items
    Demos_columns = [col for col in df.columns if col.startswith(f'Demos_')]

    # Create a copy of the relevant columns
    scored_data = df[Demos_columns].copy()

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # Add metadata as a multi-index (if applicable)
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)

    # Save the scored data
    scored_data.to_csv(path, index=False)

    return scored_data

## Demos Dummy test

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Big Five Inventory (BFI) scoring
def score_demographics_test(df, path):

    # List of Demographic items
    Demos_columns = [col for col in df.columns if col.startswith(f'Demos_')]

    # Create a copy of the relevant columns
    scored_data = df[Demos_columns].copy()

    scored_data[:] = 0

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))


    # Add metadata as a multi-index (if applicable)
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)

    # Save the scored data
    scored_data.to_csv(path,index=False)

    return scored_data

## Demographics S2

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Big Five Inventory (BFI) scoring
def score_demographics_2(df, path):

    # List of Demographic items
    Demos_columns = [col for col in df.columns if col.startswith(f'Demographics_')]

    # Create a copy of the relevant columns
    scored_data = df[Demos_columns].copy()

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))

    # Add metadata as a multi-index (if applicable)
    scored_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)

    # Save the scored data
    scored_data.to_csv(path)

    return scored_data

## S3 Demographics

In [ ]:
import pandas as pd
import hashlib

def score_demographics_s3(df, path):
    # Grab all Demographics_ and NewDemographics_ columns,
    # exclude any whose name contains "Time"
    prefixes = ('Demographics_', 'NewDemographics_')
    demographics_columns = [
        col for col in df.columns
        if col.startswith(prefixes) and 'Time' not in col
    ]

    # Copy over
    scored_data = df[demographics_columns].copy()

    # Hashed Subject_ID
    scored_data.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(hash_id)
    )

    # Session 3 metadata and save
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

## S3 Readiness

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Readiness survey scoring for Session 3
def score_readiness_S3(df, path):
    # 1) Grab all Readiness_ columns
    readiness_columns = [col for col in df.columns if col.startswith('Readiness_')]

    # Copy over
    scored_data = df[readiness_columns].copy()

    # Hashed Subject_ID
    scored_data.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(hash_id)
    )

    # Session 3 metadata & save
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

## S3 Locus of Control

In [ ]:
def score_loc(df, path):
    # Mapping of text answers to scores
    answer_scores = {
        "Rarely (less than 10% of the time)": 1,
        "Occasionally (About 30% of the time)": 2,
        "Sometimes (About half the time)": 3,
        "Frequently (About 70% of the time)": 4,
        "Usually (More than 90% of the time)": 5,
    }

    # Function to clean and map scores
    def map_scores(value):
        # Clean the string to remove leading/trailing whitespaces
        value = str(value).strip()
        # Return the mapped score or NaN if the value is not found
        return answer_scores.get(value, np.nan)

    # Function to reverse score the BFI items
    def loc_reverse_score(value):
        if pd.isna(value):
            return value  # Preserve NaN values
        return 6 - value

    # List of BFI items
    loc_columns = [f'LocusControl_{i}' for i in range(1, 29)]

    # Create a copy of the relevant columns
    scored_data = df[loc_columns].copy()

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(hash_id)
    )

    # Apply mapping and cleaning
    scored_data[loc_columns] = scored_data[loc_columns].applymap(map_scores)

    # Apply reverse scoring
    reverse_scored_items = [1, 2, 4, 6, 8, 11, 14, 17, 19, 22, 23, 24, 26, 27]
    reverse_columns = [f'LocusControl_{item}' for item in reverse_scored_items]
    scored_data[reverse_columns] = scored_data[reverse_columns].applymap(loc_reverse_score)

    # Final and additional metrics
    scored_data['LOC_Final_Score'] = scored_data[loc_columns].apply(lambda x: x.sum() if x.count() >= 1 else np.nan, axis=1)
    scored_data['LOC_Mean_of_Answers'] = scored_data[loc_columns].apply(lambda x: x.mean() if x.count() >= 1 else np.nan, axis=1)
    scored_data['LOC_SD_of_Answers'] = scored_data[loc_columns].std(axis=1)
    scored_data['LOC_Questions_Not_Answered'] = scored_data[loc_columns].isna().sum(axis=1)

    # Add metadata
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

## S3 Need for Cognition

In [ ]:
def score_nfc(df,path):
    # Mapping of text answers to scores
    answer_scores = {
        "Strongly Disagree": 1,
        "Disagree": 2,
        "Neutral": 3,
        "Agree": 4,
        "Strongly Agree": 5
    }

    # Function to clean and map scores
    def map_scores(value):
        # Clean the string to remove leading/trailing whitespaces
        value = str(value).strip()
        # Return the mapped score or NaN if the value is not found
        return answer_scores.get(value, np.nan)

    # Function to reverse score the BFI items
    def nfc_reverse_score(value):
        if pd.isna(value):
            return value  # Preserve NaN values
        return 6 - value

    # List of BFI items
    nfc_columns = [f'NeedForCognition _{i}' for i in range(1, 19)]

    # Create a copy of the relevant columns
    scored_data = df[nfc_columns].copy()

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(hash_id)
    )


    # Apply mapping and cleaning
    scored_data[nfc_columns] = scored_data[nfc_columns].applymap(map_scores)

    # Apply reverse scoring
    reverse_scored_items = [3, 4, 5, 7, 8, 9, 12, 16, 17]
    reverse_columns = [f'NeedForCognition _{item}' for item in reverse_scored_items]
    scored_data[reverse_columns] = scored_data[reverse_columns].applymap(nfc_reverse_score)

    # Final and additional metrics
    scored_data['NFC_Final_Score'] = scored_data[nfc_columns].apply(lambda x: x.sum() if x.count() >= 1 else np.nan, axis=1)
    scored_data['NFC_mean_of_Answers'] = scored_data[nfc_columns].apply(lambda x: x.mean() if x.count() >= 1 else np.nan, axis=1)
    scored_data['NFC_SD_of_Answers'] = scored_data[nfc_columns].std(axis=1)
    scored_data['NFC_Questions_Not_Answered'] = scored_data[nfc_columns].isna().sum(axis=1)

    # Add metadata
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

## S3 Emotional Reaction to Social Media Use

In [ ]:
import pandas as pd
import numpy as np
import hashlib

def score_emotion_socialmedia(df, path):
    # Mapping
    answer_scores = {
        "1 (Never)": 1,
        "2":          2,
        "3":          3,
        "4":          4,
        "5 (Always)": 5
    }
    def map_scores(value):
        return answer_scores.get(str(value).strip(), np.nan)

    # Define columns
    esm_cols = [f"EmotionSocialMedia_{i}" for i in range(1, 11)]
    n_items  = len(esm_cols)

    # Keep original header
    scored_data = df[esm_cols].copy()
    scored_data[esm_cols] = scored_data[esm_cols].applymap(map_scores)

    # Missing-value metrics
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Total score (only if ≤50% missing)
    def compute_total(row):
        return row[esm_cols].sum(min_count=1) if row['Blank_Count'] <= n_items/2 else np.nan
    scored_data['ESM_Total_Score'] = scored_data.apply(compute_total, axis=1)

    # Item-level SD
    scored_data['ESM_Item_SD'] = scored_data[esm_cols].std(axis=1, skipna=True)

    # Subscale
    pos = [f"EmotionSocialMedia_{i}" for i in [1,3,4,6,9]]
    neg = [f"EmotionSocialMedia_{i}" for i in [2,5,7,8,10]]
    scored_data['Positive_Total'] = scored_data[pos].sum(axis=1, skipna=True)
    scored_data['Negative_Total'] = scored_data[neg].sum(axis=1, skipna=True)

    # Hash ID
    scored_data.insert(
        0,
        'Subject_ID',
        df[s1_prolific_id].apply(hash_id)
    )

    # Add metadata
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

## S3 Performance Pressure

In [ ]:
import pandas as pd
import numpy as np
import hashlib

def score_performance_pressure(df, path):
    # Only these six items
    pp_cols = [
        'PerformPressure_1_1',
        'PerformPressure_2_1',
        'PerformPressure_3_1',
        'PerformPressure_4',
        'PerformPressure_5',
        'PerformPressure_6_1'
    ]

    # Copy raw responses (preserves your original headers)
    scored_data = df[pp_cols].copy()

    # Hash ID
    scored_data.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(hash_id)
    )

    # Add metadata & save
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

## S3 Flow/hyperfocusing

In [ ]:
import pandas as pd
import numpy as np
import hashlib

def score_fss_s3(df, path):
    # grab all your Flow/HypFocus columns (drop any timing ones)
    fss_cols = [c for c in df.columns
                if c.startswith('Flow/HypFocus') and 'Time' not in c]
    scored_data = df[fss_cols].copy()

    # build one big map for all the verbal labels
    text_map = {
        'Not At All':   1,
        'Partly':       4,
        'Very Much':    7,
        # Q4 scale
        'Easy':         1,
        'Average':      4,
        'Difficult':    7,
        # Q5 scale
        'Low':          1,
        'High':         7,
        # Q6 scale
        'Too Low':      1,
        'Just Right':   4,
        'Too High':     7
    }

    # define columns
    num_cols = [f'Flow/HypFocus_3_{i}' for i in range(1,14)]
    num_cols += ['Flow/HypFocus_4_1', 'Flow/HypFocus_5_1', 'Flow/HypFocus_6_1']

    # replace any of those verbal labels with numbers, then force numeric
    for col in num_cols:
        if col in scored_data:
            scored_data[col] = scored_data[col].replace(text_map)
            scored_data[col] = pd.to_numeric(scored_data[col], errors='coerce')

    # missing-data metrics on those numeric cols
    scored_data['Blank_Count']      = scored_data[num_cols].isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / len(num_cols)

    # subscale sums
    flow_items     = [f'Flow/HypFocus_3_{i}' for i in range(1,11) if f'Flow/HypFocus_3_{i}' in scored_data]
    worry_items    = [f'Flow/HypFocus_3_{i}' for i in (11,12,13)        if f'Flow/HypFocus_3_{i}' in scored_data]
    fluency_idx    = (2,4,5,7,8,9)
    absorption_idx = (1,3,6,10)
    fluency_items    = [f'Flow/HypFocus_3_{i}' for i in fluency_idx    if f'Flow/HypFocus_3_{i}' in scored_data]
    absorption_items = [f'Flow/HypFocus_3_{i}' for i in absorption_idx if f'Flow/HypFocus_3_{i}' in scored_data]

    scored_data['Flow_Total_Score']  = scored_data[flow_items].sum(axis=1, skipna=True)
    scored_data['Worry_Score']       = scored_data[worry_items].sum(axis=1, skipna=True)
    scored_data['Fluency_Score']     = scored_data[fluency_items].sum(axis=1, skipna=True)
    scored_data['Absorption_Score']  = scored_data[absorption_items].sum(axis=1, skipna=True)

    # mask out all of those metrics if they answered "No" to Q1
    no_mask = df['Flow/HypFocus_1'].astype(str).str.strip().eq('No')
    # clear out summary fields on those rows
    for c in ['Blank_Count','Blank_Percentage',
              'Flow_Total_Score','Worry_Score',
              'Fluency_Score','Absorption_Score']:
        scored_data.loc[no_mask, c] = np.nan

    # hash subject id
    scored_data.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
    )

    # add metadata header and write out
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

## S3 Object-Spatial Imagery Questionnaire (OSIQ)

In [ ]:
import pandas as pd
import numpy as np
import re
import csv
import hashlib

def score_osiq(df, path):
    # Define items
    osiq_cols = [f"OSIQ_{i}" for i in range(1, 31)]
    object_idx = {4,7,8,10,11,12,16,17,19,21,22,25,26,28,30}

    # Mapping
    answer_map = {
        "1 (totally disagree)": 1, "2": 2, "3": 3, "4": 4, "5 (totally agree)": 5
    }
    scored = df[osiq_cols].copy()
    scored[osiq_cols] = scored[osiq_cols].applymap(lambda x: answer_map.get(str(x).strip(), np.nan))

    # Reverse‐score #27
    scored["OSIQ_27"] = scored["OSIQ_27"].apply(lambda x: 6 - x if pd.notna(x) else np.nan)

    # Missing‐data metrics
    n = len(osiq_cols)
    scored["Blank_Count"]      = scored[osiq_cols].isna().sum(axis=1)
    scored["Blank_Percentage"] = scored["Blank_Count"] / n

    # Subscale means
    obj_cols = [f"OSIQ_{i}" for i in object_idx]
    spa_cols = [c for c in osiq_cols if c not in obj_cols]
    scored["OSIQ_Object_Score"]  = scored[obj_cols].mean(axis=1, skipna=True)
    scored["OSIQ_Spatial_Score"] = scored[spa_cols].mean(axis=1, skipna=True)

    # Hashed ID
    scored.insert(
        0,
        "Subject_ID",
        df[s3_prolific_id].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
    )

    # Final headers
    headers = list(scored.columns)
    type_row = []
    for h in headers:
        m = re.match(r"OSIQ_(\d+)$", h)
        if m:
            idx = int(m.group(1))
            type_row.append("Object" if idx in object_idx else "Spatial")
        else:
            type_row.append("")   # blanks for Subject_ID, Blank_Count, etc.

    # Write CSV with header + type row + data
    with open(path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerow(type_row)
        for row in scored.itertuples(index=False, name=None):
            writer.writerow(row)

    return scored

## S3 Frequency of Social Media Use

In [ ]:
import pandas as pd
import numpy as np
import csv
import hashlib

def score_freq_social_media(df, path):
    # Raw survey columns
    cols = [
        "FreqSocialMediaUse_1_1",  # Twitter/X
        "FreqSocialMediaUse_1_2",  # Instagram
        "FreqSocialMediaUse_1_3",  # Facebook
        "FreqSocialMediaUse_1_4",  # Snapchat
        "FreqSocialMediaUse_1_5",  # YouTube
        "FreqSocialMediaUse_1_7",  # Pinterest
        "FreqSocialMediaUse_1_8",  # Reddit
        "FreqSocialMediaUse_1_9",  # TikTok
        "FreqSocialMediaUse_2",    # Overall effect on society
        "FreqSocialMediaUse_3"     # Overall effect on you personally
    ]

    # Labels for all raw columns
    label_map = {
        "FreqSocialMediaUse_1_1": "Twitter/X",
        "FreqSocialMediaUse_1_2": "Instagram",
        "FreqSocialMediaUse_1_3": "Facebook",
        "FreqSocialMediaUse_1_4": "Snapchat",
        "FreqSocialMediaUse_1_5": "YouTube",
        "FreqSocialMediaUse_1_7": "Pinterest",
        "FreqSocialMediaUse_1_8": "Reddit",
        "FreqSocialMediaUse_1_9": "TikTok",
        "FreqSocialMediaUse_2":   "Effect on society",
        "FreqSocialMediaUse_3":   "Effect on you personally"
    }

    # copy raw text responses
    scored = df[cols].copy()

    # blank metrics
    n_items = len(cols)
    scored['Blank_Count']      = scored.isna().sum(axis=1)
    scored['Blank_Percentage'] = scored['Blank_Count'] / n_items

    # numeric duplicates only for the first 8 frequency items
    usage_cols = cols[:8]
    freq_map = {
        "Never": 0,
        "Less Often": 1,
        "Several Times a Week": 2,
        "About Once a Day": 3,
        "Several Times a Day": 4,
        "Almost Constantly": 5,
        "I Don’t Know What This Is": -1
    }
    def map_freq(val):
        return freq_map.get(str(val).strip(), np.nan)

    num_df = df[usage_cols].copy().applymap(map_freq)
    num_cols = [c + "_num" for c in usage_cols]
    num_df.columns = num_cols

    # append numeric block
    scored = pd.concat([scored, num_df], axis=1)

    # hash ID
    scored.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
    )

    # write CSV with two‐line header
    with open(path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)

        # header
        writer.writerow(scored.columns)

        # second row:
        #    - blank under Subject_ID
        #    - human labels under all 10 raw columns
        #    - blanks under Blank_Count & Blank_Percentage
        #    - labels under the 8 numeric columns
        second = (
            [""] +
            [label_map[c] for c in cols] +
            ["", ""] +  # blanks under the two metrics
            [label_map[c] for c in usage_cols]
        )
        writer.writerow(second)

        # data rows (nan → blank)
        for row in scored.itertuples(index=False, name=None):
            clean = ["" if pd.isna(v) else v for v in row]
            writer.writerow(clean)

    return scored

## S3 Views on AI

In [ ]:
import pandas as pd
import numpy as np
import hashlib

def score_viewsai(df, path):
    # Raw ViewsAI columns (preserves your MultiIndex question headers)
    viewsai_cols = [
        'ViewsAI_1',
        'ViewsAI_2_1', 'ViewsAI_2_2', 'ViewsAI_2_3',
        'ViewsAI_3_1', 'ViewsAI_3_2', 'ViewsAI_3_3',
        'ViewsAI_3_4', 'ViewsAI_3_5', 'ViewsAI_3_6',
        'ViewsAI_3_7', 'ViewsAI_3_8', 'ViewsAI_3_9',
        'ViewsAI_3_10', 'ViewsAI_3_11',
        'ViewsAI_5',
        'ViewsAI_6',
        'ViewsAI_7'
    ]

    # Copy raw responses
    scored_data = df[viewsai_cols].copy()

    # Missing‐data metrics over those 18 items
    n_items = len(viewsai_cols)
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # hash subject id
    scored_data.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(hash_id)
    )

    # add metadata
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

## S3 VVIQ

In [ ]:
import pandas as pd
import numpy as np
import hashlib

# Vividness of Visual Imagery Questionnaire (VVIQ) scoring for Session 3


def score_vviq_s3(df, path):
    # Define the VVIQ item columns
    vviq_cols = [f'VVIQ_{i}' for i in range(1, 17)]
    n_items = len(vviq_cols)

    # Copy responses
    scored_data = df[vviq_cols].copy()

    # Map responses to numeric 1-5
    answer_map = {
        "No image at all (only \"knowing\" that you are thinking of the object)": 1,
        "Vague and dim": 2,
        "Moderately clear and vivid": 3,
        "Clear and reasonably vivid": 4,
        "Perfectly clear and as vivid as normal vision": 5
    }
    scored_data[vviq_cols] = scored_data[vviq_cols].applymap(
        lambda x: answer_map.get(str(x).strip(), np.nan)
    )

    # Missing-value metrics
    scored_data['Blank_Count']      = scored_data.isna().sum(axis=1)
    scored_data['Blank_Percentage'] = scored_data['Blank_Count'] / n_items

    # Total score (only if ≤50% missing)
    def compute_total(row):
        return row[vviq_cols].sum(min_count=1) if row['Blank_Count'] <= n_items/2 else np.nan
    scored_data['VVIQ_Total'] = scored_data.apply(compute_total, axis=1)

    # Item-level SD
    scored_data['VVIQ_Item_SD'] = scored_data[vviq_cols].std(axis=1, skipna=True)

    # Hash Subject_ID
    scored_data.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
    )

    # Add metadata & save
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

## S3 Self-report Aphantasia

In [ ]:
import pandas as pd
import numpy as np
import hashlib

def score_aphantasia(df, path):
    # Aphantasia questions
    raw_cols = ['Aphantasia_1', 'Aphantasia_2']
    num_cols = [c + '_num' for c in raw_cols]

    # Copy raw responses
    scored = df[raw_cols].copy()

    # Map to numeric duplicates
    map1 = {
        "I have never heard of aphantasia": 4,
        "I have heard of the term, but not sure what it is": 3,
        "I think I know what aphantasia is": 2,
        "I am very familiar with aphantasia": 1
    }
    scored['Aphantasia_1_num'] = df['Aphantasia_1'].map(lambda v: map1.get(str(v).strip(), np.nan))

    map2 = {
        "I am confident I have aphantasia": 1,
        "I think I might have aphantasia": 2,
        "I think I might have normal imagery": 3,
        "I am confident I have normal mental imagery": 4
    }
    scored['Aphantasia_2_num'] = df['Aphantasia_2'].map(lambda v: map2.get(str(v).strip(), np.nan))

    # Missing‐data metrics on the numeric items
    n = len(num_cols)
    scored['Blank_Count']      = scored[num_cols].isna().sum(axis=1)
    scored['Blank_Percentage'] = scored['Blank_Count'] / n

    # Hashed ID
    scored.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
    )

    # Add metadata row and save
    scored = add_metadata_as_multiindex(scored, s3_metadata_row)
    scored.to_csv(path, index=False)
    return scored

## S3 Readiness Pt 2

In [ ]:
import pandas as pd
import hashlib

# Readiness Part 2 scoring for Session 3, excluding any “Time” columns
def score_readiness_pt2_S3(df, path):

    readinesspt2_columns = [
        col for col in df.columns
        if col.startswith('ReadinessPt2_') and 'Time' not in col
    ]

    # Copy over
    scored_data = df[readinesspt2_columns].copy()

    # Hashed Subject_ID
    scored_data.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(hash_id)
    )

    # Session 3 metadata & save
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

## S3 Attention Checks

In [ ]:
import pandas as pd
import hashlib

def score_attn_S3(df, path):
    # List of attention check items (blocks 1–3, attempts 1–4)
    attn_columns = [f'AttnCheck_{i}.{j}' for i in range(1, 4) for j in range(1, 5)]

    # Copy just those columns
    scored_data = df[attn_columns].copy()

    # Count attempts per block
    for i in range(1, 4):
        cols = [c for c in attn_columns if c.startswith(f'AttnCheck_{i}.')]
        scored_data[f'AttnCheck_{i}_attempts'] = scored_data[cols].notna().sum(axis=1)

    # Summaries
    attempt_cols = [c for c in scored_data.columns if c.endswith('_attempts')]
    # total attempts minus number of blocks touched
    total_sum = scored_data[attempt_cols].sum(axis=1)
    checks_attempted = (scored_data[attempt_cols] > 0).sum(axis=1)
    scored_data['Attn_Redo_Count']     = total_sum - checks_attempted
    # how many checks were failed (all 4 tries used)
    scored_data['Failed_Attn_Checks']  = (scored_data[attempt_cols] == 4).sum(axis=1)
    # flag if they maxed out redos
    scored_data['Attn_Max_Redos?']     = scored_data['Attn_Redo_Count'] >= 7

    # Hash Prolific ID
    scored_data.insert(
        0,
        'Subject_ID',
        df[s3_prolific_id].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
    )

    # metadata and save
    scored_data = add_metadata_as_multiindex(scored_data, s3_metadata_row)
    scored_data.to_csv(path, index=False)

    return scored_data

# Readiness S1

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Big Five Inventory (BFI) scoring
def score_readiness(df, path):

    # List of readiness items
    ##readiness_columns = [f'Readiness_{i}' for i in range(1, 17)]  # Adjust the range according to the number of items in readiness
    readiness_columns = [col for col in df.columns if col.startswith(f'Readiness_')]

    # Create a copy of the relevant columns
    scored_data = df[readiness_columns].copy()

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # Add metadata as a multi-index (if applicable)
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)

    # Save the scored data
    scored_data.to_csv(path, index=False)

    return scored_data

# Second Readiness S1

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Big Five Inventory (BFI) scoring
def score_second_readiness(df, path):

    # List of readiness items
    ##readiness_columns = [f'Readiness_{i}' for i in range(1, 17)]  # Adjust the range according to the number of items in readiness
    readiness_columns = [col for col in df.columns if col.startswith(f'Second_Readiness')]

    # Create a copy of the relevant columns
    scored_data = df[readiness_columns].copy()

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # Add metadata as a multi-index (if applicable)
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)

    # Save the scored data
    scored_data.to_csv(path, index=False)

    return scored_data

# Readiness S2

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Big Five Inventory (BFI) scoring
def score_readiness_S2(df, path):

    # List of readiness items
    ##readiness_columns = [f'Readiness_{i}' for i in range(1, 17)]  # Adjust the range according to the number of items in readiness
    readiness_columns = [col for col in df.columns if col.startswith(f'Readiness_')]

    # Create a copy of the relevant columns
    scored_data = df[readiness_columns].copy()

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))

    # Add metadata as a multi-index (if applicable)
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)

    # Save the scored data
    scored_data.to_csv(path,index=False)

    return scored_data

# S2 Second Readiness

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from google.colab import drive

# Big Five Inventory (BFI) scoring
def score_second_readiness_S2(df, path):

    # List of readiness items
    ##readiness_columns = [f'Readiness_{i}' for i in range(1, 17)]  # Adjust the range according to the number of items in readiness
    readiness_columns = [col for col in df.columns if col.startswith(f'ReadinessPt2_')]

    # Create a copy of the relevant columns
    scored_data = df[readiness_columns].copy()

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))

    # Add metadata as a multi-index (if applicable)
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)

    # Save the scored data
    scored_data.to_csv(path, index=False)

    return scored_data

# Attention Checks

In [ ]:
## Need to save column for each of the attention check questions
## Subject number and how many tries from each attention check

# Attention check (attn) scoring
def score_attn_S1(df, path):

# List of attention check items
    attn_columns = [f'Attn_Check_{i}_{j}' for i in range(1, 4) for j in range(1,5) ]  # Adjust the range according to the number of items in attn checks

    # Create a copy of the relevant columns
    scored_data = df[attn_columns].copy()

    # Create a dictionary to hold the counts
    attn_check_counts = {}

    # Create summary columns based on the counts for each row
    for i in range(1, 4):
        cols = [col for col in scored_data.columns if col.startswith(f'Attn_Check_{i}_')]
        scored_data[f'attn_check_{i}_attempts'] = scored_data[cols].notna().sum(axis=1)

    # Summaries for attention checks
    attempt_columns = [col for col in scored_data.columns if col.startswith('attn_check_')]
    failed_checks = (scored_data[attempt_columns] == 4).sum(axis=1)

    checks_attempted = (scored_data[attempt_columns] > 0).sum(axis=1)
    total_sum = scored_data[attempt_columns].sum(axis=1)
    scored_data['Attn_Redo_Count'] = total_sum - checks_attempted
    scored_data['Failed_Attn_Checks'] = failed_checks
    scored_data['Attn_Max_Redos?'] = scored_data['Attn_Redo_Count'] >= 7

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s1_prolific_id].apply(hash_id))

    # Add metadata as a multi-index (if applicable)
    output_data = add_metadata_as_multiindex(scored_data, s1_metadata_row)

    # Save the scored data
    scored_data.to_csv(path,index=False)

    return scored_data


# S2 Attention Checks

In [ ]:
## Need to save column for each of the attention check questions
## Subject number and how many tries from each attention check

# Attention check (attn) scoring
def score_attn_S2(df, path):

# List of attention check items
    attn_columns = [f'AttnCheck{i}.{j}' for i in range(1, 4) for j in range(1,5) ]  # Adjust the range according to the number of items in attn checks

    # Create a copy of the relevant columns
    scored_data = df[attn_columns].copy()

    # Create a dictionary to hold the counts
    attn_check_counts = {}

    # Create summary columns based on the counts for each row
    for i in range(1, 4):
        cols = [col for col in scored_data.columns if col.startswith(f'AttnCheck{i}')]
        scored_data[f'AttnCheck_{i}_attempts'] = scored_data[cols].notna().sum(axis=1)

    # Summaries for attention checks
    attempt_columns = [col for col in scored_data.columns if col.startswith('attn_check_')]
    failed_checks = (scored_data[attempt_columns] == 4).sum(axis=1)

    checks_attempted = (scored_data[attempt_columns] > 0).sum(axis=1)
    total_sum = scored_data[attempt_columns].sum(axis=1)
    scored_data['Attn_Redo_Count'] = total_sum - checks_attempted
    scored_data['Failed_Attn_Checks'] = failed_checks
    scored_data['Attn_Max_Redos?'] = scored_data['Attn_Redo_Count'] >= 7

    # Add hashed `PROLIFIC_PID` to the result
    scored_data.insert(0, 'Subject_ID', df[s2_prolific_id].apply(hash_id))

    # Add metadata as a multi-index (if applicable)
    output_data = add_metadata_as_multiindex(scored_data, s2_metadata_row)

    # Save the scored data
    scored_data.to_csv(path, index=False)

    return scored_data


# Score Surveys

In [ ]:
# Session 1 Paths
aq_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_AQ.csv'
bis_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_BIS.csv'
bfi_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_BFI.csv'
adhd_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_ADHD.csv'
stai_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_STAI.csv'
ocd_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_OCD.csv'
grit_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_GRIT.csv'
oldenburg_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_OLDENBURG.csv'
maslach_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_MASLACH.csv'
attn_path = '/content/drive/My Drive/battery_survey_scoring/analyses/JG_scored_Attn_Checks.csv'
readi_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_scored_Readiness.csv'
second_readi_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_scored_Second_Readiness.csv'
demos_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_scored_Demos.csv'
demos_test_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_test_Demos.csv'
overview_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_scored_Overview.csv'
wordle_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_scored_Wordle.csv'



# Session 2 Paths
bfne_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_AL_scored_BFNE.csv'
cusados_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_AL_scored_CUSADOS.csv'
ucla_loneliness_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_AL_scored_UCLA_Loneliness.csv'
gallup_bff_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_AL_scored_Gallup_BFF.csv'
audit_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_AL_scored_AUDIT.csv'
brcs_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_AL_scored_BRCS.csv'
pastimes_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_AL_scored_PASTIMES.csv'
psqi_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_AL_scored_PSQI.csv'
mmti_s_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_AL_scored_MMTI-S.csv'
panas_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_PANAS.csv'
pfi_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_PFI.csv'
pfi_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_PFI_test.csv'
attn_s2_path = '/content/drive/My Drive/battery_survey_scoring/analyses/JG_s2_scored_Attn_Checks.csv'
readi_s2_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_s2_scored_Readiness.csv'
overview_s2_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_s2_scored_Overview.csv'
demos_s2_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_s2_scored_Demos.csv'
cudit_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_scored_CUDIT.csv'
second_readi_s2_path = '/content/drive/My Drive/battery_survey_scoring/analyses/J_s2_scored_Second_Readiness.csv'



# Session 3 Paths
esm_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_EmotionSocialMedia.csv'
perf_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_PerformancePressure.csv'
osiq_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_OSIQ.csv'
freq_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_FreqSocialMediaUse.csv'
viewsai_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_ViewsAI.csv'
aphantasia_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_Aphantasia.csv'
demo_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_S3_Demos.csv'
readi_s3_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_Readiness.csv'
readi2_s3_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_ReadinessPt2.csv'
attn_s3_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_S3_Attn_Checks.csv'
flow_s3_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_Flow.csv'
vviq_s3_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_VVIQ.csv'
loc_s3_path = '/content/drive/My Drive/battery_survey_scoring/analyses/scored_LOC.csv'
nfc_s3_path    = '/content/drive/My Drive/battery_survey_scoring/analyses/scored_NFC.csv'
overview_s3_path = '/content/drive/My Drive/battery_survey_scoring/analyses/SM_scored_Overview_S3.csv'


# Session 1 Function Execution
# scored_aq_data = score_aq(s1_filtered_data, aq_path)
# scored_bis_data = score_bis(s1_filtered_data, bis_path)
# scored_bfi_data = score_bfi(s1_filtered_data, bfi_path)
# scored_adhd_data = score_adhd(s1_filtered_data, adhd_path)
# scored_stai_data = score_stai(s1_filtered_data, stai_path)
# scored_ocd_data = score_ocd(s1_filtered_data, ocd_path)
# scored_grit_data = score_grit(s1_filtered_data, grit_path)
# scored_oldenburg_data = score_oldenburg(s1_filtered_data, oldenburg_path)
# scored_maslach_data = score_maslach(s1_filtered_data, maslach_path)
# scored_attn_data = score_attn_S1(s1_filtered_data, attn_path)
# scored_readi_data = score_readiness(s1_filtered_data, readi_path)
# scored_demos_data = score_demographics(s1_filtered_data, demos_path)
# test_scored_demos_data = score_demographics_test(s1_filtered_data, demos_test_path)
# scored_overview_data = score_overview(s1_prolific_combined_df, overview_path)
# scored_wordle_data = score_wordle(s1_filtered_data, wordle_path)
# scored_second_readi_data = score_second_readiness(s1_filtered_data, second_readi_path)



# Session 2 Function Execution
# scored_bfne_data = score_bfne(s2_filtered_data, bfne_path)
# scored_cusados_data = score_cusados(s2_filtered_data, cusados_path)
# scored_ucla_loneliness_data = score_ucla_loneliness(s2_filtered_data, ucla_loneliness_path)
# scored_gallup_bff_data = score_gallup_bff(s2_filtered_data, gallup_bff_path)
# scored_audit_data = score_audit(s2_filtered_data, audit_path)
# scored_brcs_data = score_brcs(s2_filtered_data, brcs_path)
# scored_pastimes_data = score_pastimes(s2_filtered_data, pastimes_path)
# scored_psqi_data = score_psqi(s2_filtered_data, psqi_path)
# scored_mmti_s_data = score_mmti_s(s2_filtered_data, mmti_s_path)
# scored_panas_data = score_panas(s2_filtered_data, panas_path)
# scored_pfi_data = score_pfi(s2_filtered_data, pfi_path)
# scored_pfi_data = score_pfi_with_questions_inline(s2_filtered_data, pfi_path)
# scored_s2_attn_data = score_attn_S2(s2_filtered_data, attn_s2_path)
# scored_s2_readi_data = score_readiness_S2(s2_filtered_data, readi_s2_path)
# scored_s2_overview_data = score_overview_s2(s2_prolific_data, overview_s2_path)
# scored_s2_demos_data = score_demographics_2(s2_filtered_data, demos_s2_path)
# scored_cudit_data = score_cudit(s2_filtered_data, cudit_path)
# scored_second_readi_s2_data = score_second_readiness_S2(s2_filtered_data, second_readi_s2_path)

# Session 3 Function Execution
# scored_esm_data = score_emotion_socialmedia(s3_filtered_data, esm_path)
# scored_perf_data = score_performance_pressure(s3_filtered_data, perf_path)
# scored_osiq_data = score_osiq(s3_filtered_data, osiq_path)
# scored_freq = score_freq_social_media(s3_filtered_data, freq_path)
# scored_viewsai_data = score_viewsai(s3_filtered_data, viewsai_path)
# scored_aphantasia = score_aphantasia(s3_filtered_data, aphantasia_path)
# scored_demos_s3 = score_demographics_s3(s3_filtered_data, demo_path)
# scored_readi_s3 = score_readiness_S3(s3_filtered_data, readi_s3_path)
# scored_readi_pt2_s3 = score_readiness_pt2_S3(s3_filtered_data, readi2_s3_path)
# scored_attn_s3 = score_attn_S3(s3_filtered_data, attn_s3_path)
# scored_flow_s3 = score_fss_s3(s3_filtered_data, flow_s3_path)
# scored_vviq_s3 = score_vviq_s3(s3_filtered_data, vviq_s3_path)
# scored_loc_s3 = score_loc(s3_filtered_data, loc_s3_path)
scored_nfc_s3  = score_nfc(s3_filtered_data, nfc_s3_path)
scored_overview_s3 = score_overview_s3(s3_prolific_data, overview_s3_path)

/tmp/ipython-input-1748562585.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  scored_data[nfc_columns] = scored_data[nfc_columns].applymap(map_scores)
/tmp/ipython-input-1748562585.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  scored_data[reverse_columns] = scored_data[reverse_columns].applymap(nfc_reverse_score)


In [ ]:
#scored_cudit_data

# Export to Excel Sheet with multiple tabs

In [ ]:
#S2_bfne have session be a variable, all caps for acronyms
#test extra column in old file and new file
#add session 1 data
#attention check tabs (4 questions)

# Create an Excel writer object
# with pd.ExcelWriter('/content/drive/My Drive/battery_survey_scoring/data/excel_data/battery_data.xlsx') as writer:
#     # Write each DataFrame to a different sheet
#     scored_bfne_data.to_excel(writer, sheet_name='S2_bfne', index=True)
#     scored_cusados_data.to_excel(writer, sheet_name='S2_cusados', index=True)
#     scored_ucla_loneliness_data.to_excel(writer, sheet_name='S2_loneliness', index=True)
#     scored_gallup_bff_data.to_excel(writer, sheet_name='S2_gallup', index=True)
#     scored_audit_data.to_excel(writer, sheet_name='S2_audit', index=True)
#     scored_brcs_data.to_excel(writer, sheet_name='S2_brcs', index=True)
#     scored_pastimes_data.to_excel(writer, sheet_name='S2_pastimes', index=True)
#     scored_psqi_data.to_excel(writer, sheet_name='S2_psqi', index=True)
#     scored_mmti_s_data.to_excel(writer, sheet_name='S2_mmti', index=True)
#     scored_panas_data.to_excel(writer, sheet_name='S2_panas', index=True)
#     scored_pfi_data.to_excel(writer, sheet_name='S2_pfi', index=True)

In [ ]:
!ls

drive  sample_data


# Merge New Data with Excel Sheet

This appends new rows to each tab and adds new tab while mainting the styling and enhancements to the original file

In [ ]:
# from openpyxl import load_workbook
# import pandas as pd

# # File paths
# #original_excel_path = '/content/drive/My Drive/battery_survey_scoring/data/excel_data/battery_data_test.xlsx'
# original_excel_path = '/content/drive/My Drive/battery_survey_scoring/data/excel_data/battery_data_merged_2025_06_11_SRMEdits.xlsx'
# new_excel_path = '/content/drive/My Drive/battery_survey_scoring/data/excel_data/battery_data_7_22_2025.xlsx'

# # Read new Excel file
# new_excel = pd.read_excel(new_excel_path, sheet_name=None)

# # Load the original workbook with openpyxl
# wb = load_workbook(original_excel_path)

# # Variable to track the total number of new unique Subject_IDs
# total_new_subject_ids = 0

# # Dictionary to store added Subject_ID counts for each sheet
# sheet_subject_id_counts = {}

# # Iterate through sheets
# for sheet_name, new_df in new_excel.items():
#     if sheet_name in wb.sheetnames:
#         ws = wb[sheet_name]

#         # Convert the original sheet into a DataFrame, skipping empty rows
#         original_data = list(ws.values)
#         original_df = pd.DataFrame(original_data[1:], columns=original_data[0]).dropna(how="all")

#         # Align columns between original and new data
#         new_df = new_df[original_df.columns.intersection(new_df.columns)]

#         # Identify new unique Subject_IDs in the new data
#         if "Subject_ID" in original_df.columns and "Subject_ID" in new_df.columns:
#             original_subject_ids = set(original_df["Subject_ID"].dropna().unique())
#             new_subject_ids = set(new_df["Subject_ID"].dropna().unique())
#             added_subject_ids = new_subject_ids - original_subject_ids

#             # Update the total count
#             sheet_subject_id_counts[sheet_name] = len(added_subject_ids)
#             total_new_subject_ids += len(added_subject_ids)

#             # Print details for this sheet
#             print(f"Sheet: {sheet_name}")
#             print(f"New unique Subject_IDs added: {len(added_subject_ids)}")
#         else:
#             sheet_subject_id_counts[sheet_name] = 0
#             print(f"Sheet: {sheet_name} - No Subject_ID column found.")

#         # Concatenate and remove duplicates across all columns
#         combined_df = pd.concat([original_df, new_df], ignore_index=True).drop_duplicates()

#         # Clear the sheet (except the header row) but keep its styles
#         for row in ws.iter_rows(min_row=2, max_row=ws.max_row):  # Skip header
#             for cell in row:
#                 cell.value = None

#         # Write updated data back to the sheet
#         for r_idx, row in combined_df.iterrows():
#             for c_idx, value in enumerate(row, start=1):
#                 ws.cell(row=r_idx + 2, column=c_idx, value=value)  # Start writing from row 2

#     else:
#         # Add new sheet for new data
#         new_ws = wb.create_sheet(sheet_name)
#         # Write the header first
#         for c_idx, col_name in enumerate(new_df.columns, start=1):
#             new_ws.cell(row=1, column=c_idx, value=col_name)
#         # Write the data
#         for r_idx, row in new_df.iterrows():
#             for c_idx, value in enumerate(row, start=1):
#                 new_ws.cell(row=r_idx + 2, column=c_idx, value=value)

# # Remove blank rows from all worksheets
# for ws in wb.worksheets:
#     rows_to_delete = []
#     for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
#         if all(cell.value is None or str(cell.value).strip() == '' for cell in row):
#             rows_to_delete.append(row[0].row)

#     for row_idx in reversed(rows_to_delete):
#         ws.delete_rows(row_idx)

# # Save the updated workbook
# wb.save(original_excel_path)

# # Print the total number of new unique Subject_IDs across all sheets
# print(f"\nTotal new unique Subject_IDs added across all sheets: {total_new_subject_ids}")
# for sheet_name, count in sheet_subject_id_counts.items():
#     print(f"Sheet: {sheet_name}, New Subject_IDs added: {count}")

Sarah update

In [ ]:
# from openpyxl import load_workbook
# from openpyxl.cell.cell import MergedCell
# import pandas as pd

# # File paths
# original_excel_path = '/content/drive/My Drive/battery_survey_scoring/data/excel_data/battery_data_merged_2025_06_11_SRMEdits.xlsx'
# new_excel_path      = '/content/drive/My Drive/battery_survey_scoring/data/excel_data/battery_data_7_22_2025.xlsx'
# # <— here’s the new one:
# output_excel_path   = '/content/drive/My Drive/battery_survey_scoring/data/excel_data/battery_data_merged_2025_07_22.xlsx'

# # Read all sheets from the new workbook
# new_excel = pd.read_excel(new_excel_path, sheet_name=None)

# # Load the original workbook
# wb = load_workbook(original_excel_path)

# total_new_subject_ids = 0
# sheet_subject_id_counts = {}

# for sheet_name, new_df in new_excel.items():
#     if sheet_name in wb.sheetnames:
#         ws = wb[sheet_name]

#         # Read existing sheet into DataFrame
#         original_data = list(ws.values)
#         original_df   = (
#             pd.DataFrame(original_data[1:], columns=original_data[0])
#               .dropna(how="all")
#         )

#         # Drop any duplicated columns
#         original_df = original_df.loc[:, ~original_df.columns.duplicated()]
#         new_df      = new_df.loc[:, ~new_df.columns.duplicated()]

#         # Align columns
#         common = original_df.columns.intersection(new_df.columns)
#         original_df = original_df[common]
#         new_df      = new_df[common]

#         # Count new Subject_IDs
#         if "Subject_ID" in common:
#             orig_ids = set(original_df["Subject_ID"].dropna())
#             new_ids  = set(new_df["Subject_ID"].dropna())
#             added    = new_ids - orig_ids
#             sheet_subject_id_counts[sheet_name] = len(added)
#             total_new_subject_ids += len(added)
#             print(f"Sheet: {sheet_name} — New IDs: {len(added)}")
#         else:
#             sheet_subject_id_counts[sheet_name] = 0
#             print(f"Sheet: {sheet_name} — No Subject_ID column.")

#         # Merge & dedupe
#         combined_df = pd.concat([original_df, new_df], ignore_index=True)\
#                         .drop_duplicates()

#         # Clear old rows (skip merged‐cell placeholders)
#         for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
#             for cell in row:
#                 if isinstance(cell, MergedCell):
#                     continue
#                 cell.value = None

#         # Write merged data back
#         for r_idx, row in combined_df.iterrows():
#             for c_idx, val in enumerate(row, start=1):
#                 ws.cell(row=r_idx+2, column=c_idx, value=val)

#     else:
#         # New sheet: just write it
#         new_ws = wb.create_sheet(sheet_name)
#         for c_idx, col in enumerate(new_df.columns, start=1):
#             new_ws.cell(row=1, column=c_idx, value=col)
#         for r_idx, row in new_df.iterrows():
#             for c_idx, val in enumerate(row, start=1):
#                 new_ws.cell(row=r_idx+2, column=c_idx, value=val)

# # Drop fully blank rows
# for ws in wb.worksheets:
#     blank_rows = []
#     for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
#         if all(cell.value in (None, "") for cell in row):
#             blank_rows.append(row[0].row)
#     for rid in reversed(blank_rows):
#         ws.delete_rows(rid)

# # SAVE UNDER A NEW FILENAME
# wb.save(output_excel_path)

# # Summary
# print(f"\nTotal new unique IDs added: {total_new_subject_ids}")
# for sheet, cnt in sheet_subject_id_counts.items():
#     print(f"  {sheet}: {cnt}")

# Scrap

In [ ]:
# # Initialize with PID and has_Ts_and_Ls (and other demos)
# master_df = filtered_data[['Q5', 'has_Ts_and_Ls']].copy()
# master_df['Subject_ID'] = filtered_data['Q5'].apply(hash_id)

# # Selecting specific AQ columns to merge
# aq_columns_to_merge = [
#     'AQ_Social_Skill', 'AQ_Attention_Switching', 'AQ_Attention_To_Detail',
#     'AQ_Communication', 'AQ_Imagination', 'AQ_Short_Social_Skill',
#     'AQ_Short_Routine', 'AQ_Short_Switching', 'AQ_Short_Imagination',
#     'AQ_Short_Numbers_and_Patterns', 'AQ_Three_Factor_Sociability',
#     'AQ_Three_Factor_Mentalizing', 'AQ_Three_Factor_Detail_Orientation',
#     'AQ_Total_Score', 'AQ_exclude_sparse', 'AQ_exclude_zero_std', 'Subject_ID'
# ]

# # Selecting BIS columns to merge
# bis_columns_to_merge = [
#     'BIS_Attentional_Scores','BIS_Motor_Scores','BIS_Nonplanning_Scores','BIS_Final_Score',
#     'BIS_SD_of_Answers','BIS_Questions_Not_Answered', 'Subject_ID'
# ]

# # Add yours here:

# # Merge AQ data
# master_df = master_df.merge(scored_aq_data[aq_columns_to_merge], on='Subject_ID', how='left')

# # Merge BIS data
# master_df = master_df.merge(scored_bis_data[bis_columns_to_merge], on='Subject_ID', how='left')

# # Optionally, save the master dataframe
# master_df.to_csv('/content/drive/My Drive/battery_survey_scoring/analyses/master_df.csv', index=False)

In [ ]:
# import pandas as pd
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Load survey data
# data_path = '/content/drive/My Drive/battery_survey_scoring/data/survey/Burnout Battery - Prolific (April 2024)_May 5, 2024_19.56.csv'
# data = pd.read_csv(data_path)

# # Load worker data for Ts and Ls
# worker_data_path = '/content/drive/My Drive/battery_survey_scoring/data/worker_info/Ts-and-Ls_burnout_prolific_worker_data_S1.csv'
# worker_data = pd.read_csv(worker_data_path)

# # Extract the 'workerId' column into a list
# prolific_ids = set(worker_data['workerId'].tolist())  # Use a set for faster lookup

# # Mark each survey entry with whether the ID is in the worker data
# data['has_Ts_and_Ls'] = data['Q5'].apply(lambda x: 'Yes' if x in prolific_ids else 'No')

# # Remove duplicates based on PROLIFIC_PID
# filtered_data = data.drop_duplicates(subset=['Q5'], keep='first')

# # Output the number of unique PROLIFIC_PID in the survey and those matching with worker data
# print(f"Total Prolific IDs in survey: {data['Q5'].nunique()}")
# print(f"Total unique IDs in filtered data: {filtered_data['Q5'].nunique()}")
# print(f"IDs matched with Ts and Ls data: {filtered_data['has_Ts_and_Ls'].value_counts()['Yes']}")

# import hashlib
# import pandas as pd

# def hash_id(participant_id):
#     """Hashes the participant ID using SHA-256."""
#     hash_object = hashlib.sha256()
#     hash_object.update(participant_id.encode())
#     return hash_object.hexdigest()

# # AQ Functions
# # Function to create the column names with 'ASQ_' prefix
# def asq_column(question_number):
#     return f'ASQ_{question_number}'

# # Define question sets for each scoring scheme
# default_subscales = {
#     'Social_Skill': [1, 11, 13, 15, 22, 36, 44, 45, 47, 48],
#     'Attention_Switching': [2, 4, 10, 16, 25, 32, 34, 37, 43, 46],
#     'Attention_To_Detail': [5, 6, 9, 12, 19, 23, 28, 29, 30, 49],
#     'Communication': [7, 17, 18, 26, 27, 31, 33, 35, 38, 39],
#     'Imagination': [3, 8, 14, 20, 21, 24, 40, 41, 42, 50]
# }

# asq_short_subscales = {
#     'ASQ_Short_Social_Skill': [1, 15, 36, 45, 50],
#     'ASQ_Short_Routine': [2, 25, 34, 46],
#     'ASQ_Short_Switching': [4, 10, 32, 37],
#     'ASQ_Short_Imagination': [3, 8, 14, 20, 36, 42, 45, 50],
#     'ASQ_Short_Numbers_and_Patterns': [6, 9, 19, 23, 41]
# }

# three_factor_subscales = {
#     'Three_Factor_Sociability': [7, 11, 17, 28, 31, 42, 50],
#     'Three_Factor_Mentalizing': [2, 15, 23, 29, 30, 32],
#     'Three_Factor_Detail_Orientation': [3, 5, 12, 25, 26, 33, 38]
# }

# macro_1_questions = [1, 2, 4, 5, 6, 7, 9, 12, 13, 16, 18, 19, 20, 21, 22, 23, 26, 33, 35, 39, 41, 42, 43, 45, 46]
# macro_2_questions = [3, 8, 10, 11, 14, 15, 17, 24, 25, 27, 28, 29, 30, 31, 32, 34, 36, 37, 38, 40, 44, 47, 48, 49, 50]

# def apply_asq_macros(row):
#     """Apply macros for scoring questions."""
#     scores = {}

#     for question in macro_1_questions:
#         col = asq_column(question)
#         scores[col] = int(row[col] in ["Definitely Agree", "Slightly Agree"])
#         # 1 if definitely agree or slightly agree; else 0

#     for question in macro_2_questions:
#         col = asq_column(question)
#         scores[col] = int(row[col] in ["Definitely Disagree", "Slightly Disagree"])
#         # 1 if definitely disagree or slightly disagree; else 0

#     return pd.Series(scores)

# def calculate_subscale_scores(df, subscales):
#     """Calculate subscale scores given a subscale-question mapping."""
#     for subscale_name, questions in subscales.items():
#         columns = [asq_column(q) for q in questions]
#         df[subscale_name] = df[columns].sum(axis=1)

# # Quality check functions
# def flag_sparse_data(df, question_columns, min_answers=25):
#     """Flag participants with less than the minimum number of answered questions."""
#     answered_counts = df[question_columns].notna().sum(axis=1)
#     return answered_counts < min_answers

# def flag_zero_std_data(df, question_columns):
#     """Flag participants with standard deviation of zero."""
#     return df[question_columns].std(axis=1) == 0

# # Apply macros to score each ASQ question
# scored_data = filtered_data.apply(apply_asq_macros, axis=1)

# # Flag data with less than half of the questions answered
# question_columns = [asq_column(q) for q in range(1, 51)]
# scored_data['exclude_sparse'] = flag_sparse_data(scored_data, question_columns).astype(str)

# # Flag data with no standard deviation
# scored_data['exclude_zero_std'] = flag_zero_std_data(scored_data, question_columns).astype(str)

# # Exclude flagged participants from getting subscale and total scores, but keep them in the data
# # Don't remove no standard deviation data or any of the data just use flags
# scored_data.loc[(scored_data['exclude_sparse'] == 'True') | (scored_data['exclude_zero_std'] == 'True'), scored_data.columns[scored_data.columns != 'exclude_sparse']] = None

# # Calculate subscale scores and total score for the default scoring
# calculate_subscale_scores(scored_data, default_subscales)
# scored_data['Total_Score'] = scored_data.iloc[:, :50].sum(axis=1)

# # Calculate subscale scores for the other scales
# calculate_subscale_scores(scored_data, asq_short_subscales)
# calculate_subscale_scores(scored_data, three_factor_subscales)

# # Calculate the total score
# scored_data['Total_Score'] = scored_data.iloc[:, :50].sum(axis=1)
# scored_data.loc[(scored_data['exclude_sparse'] == 'True') | (scored_data['exclude_zero_std'] == 'True'), 'Total_Score'] = None

# # Add hashed `PROLIFIC_PID` to the result
# scored_data['Subject_ID'] = filtered_data['Q5'].apply(hash_id)

# # Export the resulting DataFrame to a CSV file
# scored_data.to_csv('/content/drive/My Drive/battery_survey_scoring/analyses/ASQ_Scored_All.csv', index=False)

# print("CSV file with all ASQ scored data has been created.")

# # BIS Score

# # Mapping of text answers to scores
# answer_scores = {
#     "Rarely/Never": 1,
#     "Occasionally": 2,
#     "Often": 3,
#     "Almost always/Always": 4
# }

# # Function to clean and map scores
# def map_scores(value):
#     # Clean the string to remove leading/trailing whitespaces
#     value = str(value).strip()
#     # Return the mapped score or NaN if the value is not found
#     return answer_scores.get(value, np.nan)

# # Function to reverse score the BIS items
# def BIS_reverse_score(value):
#     if pd.isna(value):
#         return value  # Preserve NaN values
#     return 5 - value

# # Function to score the BIS survey
# def score_bis(df):
#     bis_columns = [f'BIS_{i}' for i in range(1, 31)]
#     new_df = df[bis_columns].copy()

#     # Apply mapping and cleaning
#     new_df = new_df.applymap(map_scores)

#     # Apply reverse scoring
#     reverse_scored_items = [1, 7, 8, 9, 10, 12, 13, 15, 20, 29, 30]
#     reverse_columns = [f'BIS_{item}' for item in reverse_scored_items]
#     new_df[reverse_columns] = new_df[reverse_columns].applymap(BIS_reverse_score)

#     # Compute subscale scores
#     new_df['Attentional_Scores'] = new_df[['BIS_5', 'BIS_6', 'BIS_9', 'BIS_11', 'BIS_20', 'BIS_24', 'BIS_26', 'BIS_28']].sum(axis=1, min_count=1)
#     new_df['Motor_Scores'] = new_df[['BIS_2', 'BIS_3', 'BIS_4', 'BIS_16', 'BIS_17', 'BIS_19', 'BIS_21', 'BIS_22', 'BIS_23', 'BIS_25', 'BIS_30']].sum(axis=1, min_count=1)
#     new_df['Nonplanning_Scores'] = new_df[['BIS_2', 'BIS_7', 'BIS_8', 'BIS_10', 'BIS_12', 'BIS_13', 'BIS_14', 'BIS_15', 'BIS_18', 'BIS_27', 'BIS_29']].sum(axis=1, min_count=1)

#     # Final and additional metrics
#     new_df['Final_Score'] = new_df[bis_columns].sum(axis=1, min_count=1)
#     new_df['SD_of_Answers'] = new_df[bis_columns].std(axis=1)
#     new_df['Questions_Not_Answered'] = new_df[bis_columns].isna().sum(axis=1)

#     return new_df

# # Assuming data is loaded as DataFrame named 'filtered_data'
# scored_bis_data = score_bis(filtered_data)

# # Add hashed `PROLIFIC_PID` to the result
# scored_bis_data['Subject_ID'] = filtered_data['Q5'].apply(hash_id)

# # Save the scored data
# scored_bis_data.to_csv('/content/drive/My Drive/battery_survey_scoring/analyses/scored_bis_data.csv', index=False)
